In [ ]:
import os
import re
import sys
import gzip
import time
import json
import math
import copy
import numpy
import pandas
import seaborn
import matplotlib

import numpy as np

import scipy.stats
import matplotlib.pyplot as plt

from scipy.ndimage import gaussian_filter

from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA


# seaborn.set()
matplotlib.style.use('default')
plt.axis('off')




samples = ["HG00621", "HG00741", "HG01952", "HG01978", "HG03516"]
replicates = [1, 2]
phases = ["mat", "pat"]


min_cov = 5


pipeline_name_short = ["bs", "bc", "bm", "gb", "mg"]
pipeline_name_full = ["Bismark", "BISCUIT", "bwa-meth", "gemBS", "methylGrapher"]
pipeline_full_name_lookup = dict(zip(pipeline_name_short, pipeline_name_full))
pipeline_color_lookup = {
    "bs": "#0d1780",
    "bc": "#76b5c5",
    "bm": "#873e23",
    "gb": "#b7a1f9",
    "mg": "#f0aa4f"
}


individual_experiment_tissue_label_short = {
    'ENC001': {
        1: 'spleen',
        2: 'skin',
        3: 'testis',
        4: 'thyroid gland',
        5: 'pancreas',
        6: 'lung',
    },
    'ENC002': {},
    'ENC003': {},
    'ENC004': {},
}






In [ ]:
# Common tools


def methylc_reader(fp):
    res = {}
    fh = None
    
    if fp.lower().endswith(".gz") or fp.lower().endswith(".gzip"):
        fh = gzip.open(fp, "rt")
    else:
        fh = open(fp)
    for l in fh:
        l = l.strip().split("\t")
        chrom, start, end, context, ml, strand, cov = l
        start = int(start)
        ml = float(ml)
        cov = int(cov)
        
        met = int(round(ml * cov))
        
        if chrom not in res:
            res[chrom] = {}
        res[chrom][start] = (met, cov)
    
    fh.close()
    return res


def methyl_reader(fp):
    res = {}
    fh = open(fp)
    for l in fh:
        l = l.strip().split("\t")
        segID, pos, strand, context, unmet, met, cov, ml = l
        
        pos = int(pos)
        met = int(met)
        cov = int(cov)
        
        if segID not in res:
            res[segID] = {}
        res[segID][pos] = (met, cov)
        
    fh.close()
    
    return res


def methylc_path(aligner, level, sample, replicate):
    assert aligner in pipeline_name_short
    assert level in ["cpg", "cytosine"]
    assert sample in samples
    assert replicate in replicates

    res = None
    if aligner == "bs":
        res = f"/scratch/wzhang/projects/ggWGBS/bismark_hg38_default/{sample}/{replicate}/track.{level}.methylc.gz"
    elif aligner == "bc":
        res = f"/scratch/wzhang/projects/ggWGBS/biscuit_run/{sample}/{replicate}/track.{level}.methylc.gz"
    elif aligner == "bm":
        res = f"/scratch/wzhang/projects/ggWGBS/bwameth/{sample}/{replicate}/track.{level}.methylc.gz"
    elif aligner == "gb":
        res = f"/scratch/wzhang/projects/ggWGBS/gembs_run/run_merged/8tracks/{sample}_{replicate}/track.{level}.methylc.gz"
    elif aligner == "mg":
        #assert level == "cytosine"
        res = f"/scratch/wzhang/projects/ggWGBS/methylgrapher_no_trim/surjection/{sample}_{replicate}.methylc.gz"

    # assert res is not None
    # assert os.path.exists(res)
    return res



def methylc_getter(aligner, level, sample, replicate):
    methylc_p = methylc_path(aligner, level, sample, replicate)
    if not os.path.exists(methylc_p):
        return None
    return methylc_reader(methylc_p)

for s in samples:
    for r in replicates:
        for p in pipeline_name_short:
            for level in ["cpg", "cytosine"]:
                methylc_p = methylc_path(p, level, s, r)
                if not os.path.exists(methylc_p):
                    print("ERROR", s, r, p, level, methylc_p)



        

In [ ]:
# Just reading in necessary data

bismark_cpg_data = {}
methylgrapher_lifted_cytosine_data = {}
methylgrapher_cpg_data = {}



for s in samples:
    bismark_cpg_data[s] = {}
    methylgrapher_lifted_cytosine_data[s] = {}
    methylgrapher_cpg_data[s] = {}
    
    for r in replicates:
        print(s, r)
        
        
        
        bs_fp = f"/scratch/wzhang/projects/ggWGBS/data/bismark_hg38/{s}_{r}.cpg.methylc"
        mg_fp = f"/scratch/wzhang/projects/ggWGBS/methylgrapher_no_trim/{s}/{r}/mq10.graph.CG.merged.tsv"
        
        mg2l_fp = f"/scratch/wzhang/projects/ggWGBS/methylgrapher_no_trim/surjection/{s}_{r}.methylc.gz"
        
        d = {}
        for l in open(mg_fp):
            l = l.strip().split("\t")
            cpg_ID, met, cov = l
            met = int(met)
            cov = int(cov)
            d[cpg_ID] = (met, cov)
            
            
        methylgrapher_cpg_data[s][r] = d
        

        
        # bismark_cpg_data[s][r] = methylc_reader(bs_fp)
        methylgrapher_lifted_cytosine_data[s][r] = methylc_reader(mg2l_fp)
        
        




In [ ]:
# Just reading in necessary data for Figure 5A

bismark_cpg_data = {}
methylgrapher_cytosine_data = {}



for s in samples:
    bismark_cpg_data[s] = {}
    methylgrapher_cytosine_data[s] = {}
    
    for r in replicates:
        print(s, r)

        bs_fp = f"/scratch/wzhang/projects/ggWGBS/data/bismark_hg38/{s}_{r}.cpg.methylc"
        mg_fp = f"/scratch/wzhang/projects/ggWGBS/methylgrapher_no_trim/{s}/{r}/mq10.graph.CG.methyl"
            
        methylgrapher_cytosine_data[s][r] = methyl_reader(mg_fp)
        bismark_cpg_data[s][r] = methylc_reader(bs_fp)
        
        



In [ ]:
# Figure 2A and 2B


fig2AB, axes2AB = plt.subplots(1, 2, figsize=(10, 4))


slurm_str = """


400            1.sbatch         20        10G   22:39:40   01:07:59                                   0:0 /scratch/wzhang/pro+        guinness
400.batch         batch         20              22:39:40   01:07:59    199140K    932700K                                             guinness
402            1.sbatch         20        10G   22:38:00   01:07:54                                   0:0 /scratch/wzhang/pro+        guinness
402.batch         batch         20              22:38:00   01:07:54    195192K    929660K                                             guinness
403            1.sbatch         20        10G   22:38:40   01:07:56                                   0:0 /scratch/wzhang/pro+        guinness
403.batch         batch         20              22:38:40   01:07:56    199800K    932888K                                             guinness


404            2.sbatch         20       150G   14:15:00   00:42:45                                   0:0 /scratch/wzhang/pro+          maotai
404.batch         batch         20              14:15:00   00:42:45 142916088K 142921640K                                               maotai
405            2.sbatch         20       150G   12:50:20   00:38:31                                   0:0 /scratch/wzhang/pro+          maotai
405.batch         batch         20              12:50:20   00:38:31 142916396K 142921640K                                               maotai
406            2.sbatch         20       150G   12:47:00   00:38:21                                   0:0 /scratch/wzhang/pro+          maotai
406.batch         batch         20              12:47:00   00:38:21 142916688K 142921644K                                               maotai




420          3.bismark+         20       300G 6-02:55:00   07:20:45                                   0:0 /scratch/wzhang/pro+          maotai
420.batch         batch         20            6-02:55:00   07:20:45 274326708K 294423504K                                               maotai
421          3.bismark+         20       300G 6-06:15:00   07:30:45                                   0:0 /scratch/wzhang/pro+          maotai
421.batch         batch         20            6-06:15:00   07:30:45 292913100K 312429924K                                               maotai
422          3.bismark+         20       300G 6-00:03:00   07:12:09                                   0:0 /scratch/wzhang/pro+          maotai
422.batch         batch         20            6-00:03:00   07:12:09 287908660K 309915044K                                               maotai



720          3.biscuit+         20       100G 4-12:51:00   05:26:33                                   0:0 /scratch/wzhang/pro+        guinness
720.batch         batch         20            4-12:51:00   05:26:33  38111276K  42363188K                                             guinness
741          3.biscuit+         20       100G 4-12:41:00   05:26:03                                   0:0 /scratch/wzhang/pro+        guinness 
741.batch         batch         20            4-12:41:00   05:26:03  38102880K  42428728K                                             guinness
742          3.biscuit+         20       100G 4-11:59:40   05:23:59                                   0:0 /scratch/wzhang/pro+        guinness 
742.batch         batch         20            4-11:59:40   05:23:59  38090708K  42690832K                                             guinness

700          3.bwameth+         20       100G 5-02:46:00   06:08:18                                   0:0 /scratch/wzhang/pro+        guinness
700.batch         batch         20            5-02:46:00   06:08:18  36993548K  41704288K                                             guinness
707          3.bwameth+         20       100G 5-02:09:00   06:06:27                                   0:0 /scratch/wzhang/pro+        guinness
707.batch         batch         20            5-02:09:00   06:06:27  37025940K  41769848K                                             guinness
708          3.bwameth+         20       100G 5-01:27:00   06:04:21                                   0:0 /scratch/wzhang/pro+        guinness
708.batch         batch         20            5-01:27:00   06:04:21  37003088K  41573232K                                             guinness


706          3.gembs.s+         20       100G 1-12:51:40   01:50:35                                   0:0 /scratch/wzhang/pro+          maotai
706.batch         batch         20            1-12:51:40   01:50:35  65610280K  70912712K                                               maotai
716          3.gembs.s+         20       100G 1-12:45:40   01:50:17                                   0:0 /scratch/wzhang/pro+          maotai
716.batch         batch         20            1-12:45:40   01:50:17  65232744K  70540824K                                               maotai
714          3.gembs.s+         20       100G 1-12:57:40   01:50:53                                   0:0 /scratch/wzhang/pro+          maotai
714.batch         batch         20            1-12:57:40   01:50:53  65880844K  71135264K                                               maotai



727          3.methylg+         20       100G 4-11:00:40   05:21:02                                   0:0 /scratch/wzhang/pro+        tsingtao
727.batch         batch         20            4-11:00:40   05:21:02  81600544K 134626188K                                             tsingtao
728          3.methylg+         20       100G 4-10:34:20   05:19:43                                   0:0 /scratch/wzhang/pro+        tsingtao
728.batch         batch         20            4-10:34:20   05:19:43  81634320K 135462160K                                             tsingtao
729          3.methylg+         20       100G 4-10:35:20   05:19:46                                   0:0 /scratch/wzhang/pro+        tsingtao
729.batch         batch         20            4-10:35:20   05:19:46  81565756K 134641544K                                             tsingtao



410          4.bismark+         20       100G 1-06:59:40   01:32:59                                   0:0 /scratch/wzhang/pro+          maotai
410.batch         batch         20            1-06:59:40   01:32:59   2735436K  63275696K                                               maotai
411          4.bismark+         20       100G   21:18:20   01:03:55                                   0:0 /scratch/wzhang/pro+          maotai
411.batch         batch         20              21:18:20   01:03:55   2810456K  63275700K                                               maotai
412          4.bismark+         20       100G   21:08:00   01:03:24                                   0:0 /scratch/wzhang/pro+          maotai
412.batch         batch         20              21:08:00   01:03:24   2859192K  63275636K                                               maotai



701          4.biscuit+         20       100G 1-07:00:20   01:33:01                                   0:0 /scratch/wzhang/pro+          maotai
701.batch         batch         20            1-07:00:20   01:33:01   6537504K   8362704K                                               maotai
704          4.biscuit+         20       100G 1-09:50:00   01:41:30                                   0:0 /scratch/wzhang/pro+          maotai
704.batch         batch         20            1-09:50:00   01:41:30   4570948K   6018140K                                               maotai
718          4.biscuit+         20       100G 1-09:33:20   01:40:40                                   0:0 /scratch/wzhang/pro+          maotai
718.batch         batch         20            1-09:33:20   01:40:40   1226048K   1957180K                                               maotai


709          4.bwameth+         20       100G   01:45:40   00:05:17                                   0:0 /scratch/wzhang/pro+          maotai
709.batch         batch         20              01:45:40   00:05:17    823572K   2100924K                                               maotai
710          4.bwameth+         20       100G   01:43:20   00:05:10                                   0:0 /scratch/wzhang/pro+          maotai
710.batch         batch         20              01:43:20   00:05:10    820912K   2103784K                                               maotai
711          4.bwameth+         20       100G   01:45:20   00:05:16                                   0:0 /scratch/wzhang/pro+          maotai
711.batch         batch         20              01:45:20   00:05:16    821544K   2100924K                                               maotai


712          4.gembs.s+         20       100G   09:38:20   00:28:55                                   0:0 /scratch/wzhang/pro+          maotai
712.batch         batch         20              09:38:20   00:28:55   2200184K  15857552K                                               maotai
715          4.gembs.s+         20       100G   09:43:20   00:29:10                                   0:0 /scratch/wzhang/pro+          maotai
715.batch         batch         20              09:43:20   00:29:10   2485636K  15817576K                                               maotai
717          4.gembs.s+         20       100G   09:47:00   00:29:21                                   0:0 /scratch/wzhang/pro+          maotai
717.batch         batch         20              09:47:00   00:29:21   2471200K  15817812K                                               maotai




733          4.methylg+         20       100G 2-06:57:00   02:44:51                                   0:0 /scratch/wzhang/pro+       yuengling
733.batch         batch         20            2-06:57:00   02:44:51  33076892K  33489496K                                            yuengling
735          4.methylg+         20       100G 2-05:49:40   02:41:29                                   0:0 /scratch/wzhang/pro+       yuengling
735.batch         batch         20            2-05:49:40   02:41:29  33076808K  33489512K                                            yuengling
736          4.methylg+         20       100G 2-06:01:20   02:42:04                                   0:0 /scratch/wzhang/pro+       yuengling
736.batch         batch         20            2-06:01:20   02:42:04  33077008K  33490520K                                            yuengling



""".strip().split("\n")





job_detail = {}
for l in slurm_str:
    l = l.strip().split()
    if len(l) == 0:
        continue

    job_id = l[0].split(".")[0]
    job_name = l[1]


    # print(job_id, job_name)

    if job_id not in job_detail:
        job_detail[job_id] = [None, None, None, None]

    if job_name != "batch":
        step = int(job_name.split(".")[0])
        job_detail[job_id][0] = step

        if step >= 3:
            pipeline_name = job_name.split(".")[1]
            pns = None
            if pipeline_name.startswith("bismark"):
                pns = "bs"
            elif pipeline_name.startswith("biscuit"):
                pns = "bc"
            elif pipeline_name.startswith("bwameth"):
                pns = "bm"
            elif pipeline_name.startswith("gembs"):
                pns = "gb"
            elif pipeline_name.startswith("methylg"):
                pns = "mg"
            job_detail[job_id][1] = pns
            # print(pns)
        # Figure out the aligner
        # print(job_id, step)

    if job_name == "batch":
        walltime = l[4]
        h, m, s = walltime.split(":")
        walltime = int(h) * 3600 + int(m) * 60 + int(s)
        job_detail[job_id][2] = walltime / 3600

        rss = int(l[5][:-1]) / 1024 / 1024

        job_detail[job_id][3] = rss



runtime = {}
peak_memory = {}

for p in pipeline_name_short:
    runtime[p] = [[] for _ in range(5)]
    peak_memory[p] = [[] for _ in range(5)]
    
    runtime[p][0] = [0, 0, 0]
    peak_memory[p][0] = [0, 0, 0]

for jid in job_detail:
    step, pns, rt, pm = job_detail[jid]
    # print(jid, step, pipeline, runtime, peak_memory)
    
    if step in [1, 2]:
        for p in pipeline_name_short:
            runtime[p][step].append(rt)
            peak_memory[p][step].append(pm)
            
    else:
        runtime[pns][step].append(rt)
        peak_memory[pns][step].append(pm)


for p in pipeline_name_short:
    rt_total = numpy.zeros(3)
    for i in range(1, 5):
        rt_total += runtime[p][i]
    
    runtime[p][0] = list(rt_total)



for p in pipeline_name_short:
    plot_x = numpy.arange(5) + 0.15 * (pipeline_name_short.index(p)-2)
    plot_y = []
    for i in range(5):
        print(i, p, numpy.mean(runtime[p][i]))
        plot_y.append(numpy.mean(runtime[p][i]))
    print()
    
    axes2AB[0].bar(plot_x, plot_y, width=0.15, label=pipeline_full_name_lookup[p], color=pipeline_color_lookup[p])
    axes2AB[0].errorbar(plot_x, plot_y, yerr=[numpy.std(runtime[p][i]) for i in range(5)], fmt='_', color='gray', capsize=2)


for p in pipeline_name_short:
    plot_x = numpy.arange(4) + 0.15 * (pipeline_name_short.index(p)-2)
    plot_y = []
    for i in range(1, 5):
        print(i, p, numpy.mean(peak_memory[p][i]))
        plot_y.append(numpy.mean(peak_memory[p][i]))
    print()
    
    axes2AB[1].bar(plot_x, plot_y, width=0.15, label=pipeline_full_name_lookup[p], color=pipeline_color_lookup[p])
    axes2AB[1].errorbar(plot_x, plot_y, yerr=[numpy.std(peak_memory[p][i]) for i in range(1,5)], fmt='_', color='gray', capsize=2)



axes2AB[0].set_title("Runtime (hour)")
axes2AB[0].set_xticks(numpy.arange(5))
axes2AB[0].set_xticklabels(["Total", "Trim", "Dedup", "Align", "Extract"])

axes2AB[1].set_title("Peak Memory (GB)")
axes2AB[1].set_xticks(numpy.arange(4))
axes2AB[1].set_xticklabels(["Trim", "Dedup", "Align", "Extract"])


for ax in axes2AB:
    ax.set_xticks(numpy.arange(5))
    
    ax.set_xlabel("Steps")
    ax.legend()




fig2AB.savefig(f"./figures/2AB.pdf", bbox_inches='tight')





In [ ]:
# Figure 2C

read_fate_bs = {}
read_fate_mg = {}


# This table is generated from wc -l of fastq files
# Therefore, number of read pairs should be divided by 4, number of reads should be divided by 2
read_fate_common = """HG00621 1       632425760       623983088       593210520
HG00621 2       693368864       691954268       653372836
HG00741 1       828587944       825623948       785028916
HG00741 2       669297276       666649168       638223296
HG01952 1       761988944       759292520       740328588
HG01952 2       842572440       825532616       784084512
HG01978 1       762254308       755087848       734933272
HG01978 2       1022644704      1020912880      950967944
HG03516 1       1077846324      1074625584      996966416
HG03516 2       1131705808      1128505180      1027927044""".strip().split("\n")


# Common read fate for all methods
common_read_fate = {}
for rfl in read_fate_common:
    l = rfl.strip().split()
    s, r, raw_reads, after_trim, after_dedup = l
    raw_reads = int(int(raw_reads) / 2)
    after_trim = int(int(after_trim) / 2)
    after_dedup = int(int(after_dedup) / 2)
    
    trim = raw_reads - after_trim
    dedup = after_trim - after_dedup
    
    if s not in common_read_fate:
        common_read_fate[s] = {}
    common_read_fate[s][r] = [raw_reads, trim, dedup, after_dedup]


mapping_data = {}

# 5 sample, 2 replicate, 5 methods = 50 plots
# The mapq distribution plot for aligned reads
fig2X, axes2X = plt.subplots(5, 10, figsize=(40, 20))
with open("./bam_stat_simple/summary.txt") as fh:
    
    plot_i = -1
    max_mq = 0
    for l in fh:
        plot_i += 1
        
        axi = plot_i % 5
        axj = plot_i // 5
        ax = axes2X[axi][axj]
        
        l = l.strip().split("\t", 4)
        s, r, psn, mapq_distribution = l
        
        if s not in mapping_data:
            mapping_data[s] = {}
        if r not in mapping_data[s]:
            mapping_data[s][r] = {}
        mapping_data[s][r][psn] = [0, 0]
        
        # print(s, r, psn, axi, axj)
        mapq_distribution = json.loads(mapq_distribution)
        
        bar = []
        for mq in range(255):
            mqs = str(mq)
            
            count = mapq_distribution.get(mqs, 0)
            mapping_data[s][r][psn][1] += count
            if mq > 10:
                mapping_data[s][r][psn][0] += count
            
            
            bar.append(mapq_distribution.get(mqs, 0))
            
            if count > 0 and mq > max_mq:
                max_mq = mq
            
            
        
        ax.bar(numpy.arange(255), bar)
        ax.set_title(f"{s}_{r}_{psn}")

    for axi in range(10):
        for axj in range(5):
            ax = axes2X[axj][axi]
            ax.set_xticks(numpy.arange(-10, 70, 10))
            ax.set_xlim(-5, 65)
            
            
            
        

fig2X.savefig(f"./figures/2X.pdf", bbox_inches='tight')
        




# Old style
fig2C, axes2C = plt.subplots(1, 2, sharex=True, figsize=(10, 4))

mapping_percentage = {}
mapping_percentage_mapq10 = {}
for s in samples:
    for r in replicates:
        r = str(r)
        
        after_dedup = common_read_fate[s][r][3]
        for psn in pipeline_name_short:
            
            map_count = mapping_data[s][r][psn][1]
            map_count_mapq10 = mapping_data[s][r][psn][0]
            
            map_percentage = map_count / after_dedup * 100
            map_percentage_mapq10 = map_count_mapq10 / after_dedup * 100
            
            if psn not in mapping_percentage:
                mapping_percentage[psn] = []
                mapping_percentage_mapq10[psn] = []
            
            mapping_percentage[psn].append(map_percentage)
            mapping_percentage_mapq10[psn].append(map_percentage_mapq10)


for psn in pipeline_name_short:
    plot_x = [pipeline_name_short.index(psn)] * 10
    for x in range(10):
        plot_x[x] += (x-10) * 0.03
    ploy_y1 = mapping_percentage[psn]
    ploy_y2 = mapping_percentage_mapq10[psn]

    plot_y1_ave = numpy.mean(ploy_y1)
    plot_y2_ave = numpy.mean(ploy_y2)
    
    axes2C[0].scatter(plot_x, ploy_y1, label=pipeline_full_name_lookup[psn], color=pipeline_color_lookup[psn], s=0.5)
    axes2C[1].scatter(plot_x, ploy_y2, label=pipeline_full_name_lookup[psn], color=pipeline_color_lookup[psn], s=0.5)

    # line half width
    axes2C[0].plot([numpy.min(plot_x)-0.1, numpy.max(plot_x)+0.1], [plot_y1_ave, plot_y1_ave], color=pipeline_color_lookup[psn], alpha=0.5, linewidth=1.)
    axes2C[1].plot([numpy.min(plot_x)-0.1, numpy.max(plot_x)+0.1], [plot_y2_ave, plot_y2_ave], color=pipeline_color_lookup[psn], alpha=0.5, linewidth=1.)

    axes2C[0].text(numpy.mean(plot_x)-0.25, plot_y1_ave-7, f"{plot_y1_ave:.1f}%", color="black")
    axes2C[1].text(numpy.mean(plot_x)-0.25, plot_y2_ave-7, f"{plot_y2_ave:.1f}%", color="black")



axes2C[0].set_title("Mapping Percentage")
axes2C[1].set_title("Mapping Percentage (MapQ > 10)")
for ax in axes2C:
    ax.set_xticks(numpy.arange(5))
    ax.set_xticklabels([pipeline_full_name_lookup[p] for p in pipeline_name_short])
    ax.set_ylabel("Mapping Percentage (%)")
    ax.set_ylim(0, 110)
    ax.set_yticks(range(0, 110, 20))
    
    # rotate label
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
        tick.set_fontsize(8)
    # ax.legend()
            



fig2C.savefig(f"./figures/2C.pdf", bbox_inches='tight')




fig2C, axes2C = plt.subplots(1, 2, sharex=True, figsize=(10, 4))

for psn in pipeline_name_short:
    plot_x = [pipeline_name_short.index(psn)] * 10
    ploy_y1 = mapping_percentage[psn]
    ploy_y2 = mapping_percentage_mapq10[psn]

    plot_y1_ave = numpy.mean(ploy_y1)
    plot_y2_ave = numpy.mean(ploy_y2)

    # axes2C[0].boxplot([pipeline_name_short.index(psn)], [ploy_y1], widths=0.03, patch_artist=True, boxprops=dict(facecolor=pipeline_color_lookup[psn]))
    # axes2C[1].box(ploy_y1, color=pipeline_color_lookup[psn], alpha=0.5, linewidth=1.)

    # Plot y1 in seaborn violin plot
    seaborn.violinplot(x=plot_x, y=ploy_y1, ax=axes2C[0], color=pipeline_color_lookup[psn], linewidth=1., width=0.6)
    seaborn.violinplot(x=plot_x, y=ploy_y2, ax=axes2C[1], color=pipeline_color_lookup[psn], linewidth=1., width=0.6)


    y1_pos = numpy.min(ploy_y1) - 12
    y2_pos = numpy.min(ploy_y2) - 12

    if y1_pos > 60:
        y1_pos += 4

    if y2_pos > 60:
        y2_pos += 4


    axes2C[0].text(numpy.mean(plot_x)-0.25, y1_pos, f"{plot_y1_ave:.1f}%", color="black")
    axes2C[1].text(numpy.mean(plot_x)-0.25, y2_pos, f"{plot_y2_ave:.1f}%", color="black")

axes2C[0].set_title("Mapping Percentage")
axes2C[1].set_title("Mapping Percentage (MapQ > 10)")
for ax in axes2C:
    ax.set_xticks(numpy.arange(5))
    ax.set_xticklabels([pipeline_full_name_lookup[p] for p in pipeline_name_short])
    ax.set_ylabel("Mapping Percentage (%)")
    ax.set_ylim(0, 110)
    ax.set_yticks(range(0, 110, 20))

    # rotate label
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
        tick.set_fontsize(8)
    # ax.legend()


fig2C.savefig(f"./figures/2C.pdf", bbox_inches='tight')


In [ ]:
# get methylation data
# methylation distribution genome wide
# CpG count
# CpG coverage
# CpG methylation rate




def general_table_opener(fp):
    fh = None
    if fp.lower().endswith(".gz") or fp.lower().endswith(".gzip"):
        fh = gzip.open(fp, "rt")
    else:
        fh = open(fp, "r")

    lc = 0
    for l in fh:
        yield l.strip()
        lc += 1
        if lc > 1e50:
            break

    fh.close()


def methylc_reader(fp):
    res = {}
    for l in general_table_opener(fp):
        l = l.split("\t")
        chrom, start, end, context, ml, strand, cov = l
        start = int(start)
        ml = float(ml)
        cov = int(cov)
        met = int(ml * cov)
        if chrom not in res:
            res[chrom] = {}

        res[chrom][start] = [met, cov]

    return res

def biscuit_reader(fp):
    res = {}
    for l in general_table_opener(fp):
        l = l.split("\t")
        chrom, start, end, ml, cov, other = l
        start = int(start)
        ml = float(ml)
        cov = int(cov)
        met = int(ml * cov)
        if chrom not in res:
            res[chrom] = {}

        res[chrom][start] = [met, cov]

    return res

def bwameth_reader(fp):
    res = {}
    for l in general_table_opener(fp):
        l = l.split("\t")
        if len(l) != 6:
            continue

        chrom, start, end, ml, met, unmet = l
        start = int(start)
        met = int(met)
        unmet = int(unmet)
        cov = met + unmet

        if chrom not in res:
            res[chrom] = {}

        res[chrom][start] = [met, cov]

    return res



def methylgrapher_reader(fp):
    res_ml = []
    res_cov = []
    for l in general_table_opener(fp):
        cpgid, met, cov = l.split("\t")
        met = int(met)
        cov = int(cov)

        res_ml.append(met / cov)
        res_cov.append(cov)

    res_cov = cov2freq(res_cov)
    # not including 5
    cpg_count_greater_than_cov5 = sum(res_cov[5:])
    return res_ml, res_cov, cpg_count_greater_than_cov5


def cov2freq(cov, max_cov=30):
    res = [0] * max_cov
    for c in cov:
        if c > 30:
            c = 30
        res[c-1] += 1
    return res


def linear_data_to_ml_cov(data):
    res_ml = []
    res_cov = []
    for chrom in data:
        for start in data[chrom]:
            met, c = data[chrom][start]
            res_ml.append(met/c)

            res_cov.append(c)

    res_cov = cov2freq(res_cov)

    # not including 5
    cpg_count_greater_than_cov5 = sum(res_cov[5:])
    return res_ml, res_cov, cpg_count_greater_than_cov5





fig3ab, axes3ab = plt.subplots(1, 2, figsize=(12, 5))
fig3a, axes3a = plt.subplots(2, 5, figsize=(22, 8), sharex=True)
fig3b, axes3b = plt.subplots(2, 5, figsize=(22, 8), sharex=True)



for s in samples:
    for r in replicates:
        # print(s, r)

        si = samples.index(s)
        ri = replicates.index(r)
        ax3a = axes3a[ri, si]
        ax3b = axes3b[ri, si]

        bismark_out_fp = f"/scratch/wzhang/projects/ggWGBS/bismark_hg38_default/{s}/{r}/track.cpg.methylc.gz"
        biscuit_out_fp = f"/scratch/wzhang/projects/ggWGBS/biscuit_run/{s}/{r}/my_methylation_data.merged.bed.gz"
        bwamem_out_fp = f"/scratch/wzhang/projects/ggWGBS/bwameth/{s}/{r}/methylation.cpg.bedGraph"
        gembs_out_fp = f"/scratch/wzhang/projects/ggWGBS/gembs_run/run_merged/8tracks/{s}_{r}/track.cpg.methylc.gz"
        methylgrapher_out_fp = f"/scratch/wzhang/projects/ggWGBS/methylgrapher_no_trim/{s}/{r}/mq10.graph.CG.merged.tsv"


        bismark_ml = []
        biscuit_ml = []
        bwamem_ml = []
        gembs_ml = []
        methylgrapher_ml = []

        bismark_cov = []
        biscuit_cov = []
        bwamem_cov = []
        gembs_cov = []
        methylgrapher_cov = []


        bismark_cpg_data = methylc_reader(bismark_out_fp)
        bismark_ml, bismark_cov, bs_cpg_count_cov_greater_than_5 = linear_data_to_ml_cov(bismark_cpg_data)

        biscuit_cpg_data = biscuit_reader(biscuit_out_fp)
        biscuit_ml, biscuit_cov, bc_cpg_count_cov_greater_than_5 = linear_data_to_ml_cov(biscuit_cpg_data)

        bwamem_cpg_data = bwameth_reader(bwamem_out_fp)
        bwamem_ml, bwamem_cov, bm_cpg_count_cov_greater_than_5 = linear_data_to_ml_cov(bwamem_cpg_data)

        gembs_cpg_data = methylc_reader(gembs_out_fp)
        gembs_ml, gembs_cov, gb_cpg_count_cov_greater_than_5 = linear_data_to_ml_cov(gembs_cpg_data)

        methylgrapher_ml, methylgrapher_cov, mg_cpg_count_cov_greater_than_5 = methylgrapher_reader(methylgrapher_out_fp)

        print(s, r, "Cov5", bs_cpg_count_cov_greater_than_5, bc_cpg_count_cov_greater_than_5, bm_cpg_count_cov_greater_than_5, gb_cpg_count_cov_greater_than_5, mg_cpg_count_cov_greater_than_5)
        print(s, r, "All", len(bismark_ml), len(biscuit_ml), len(bwamem_ml), len(gembs_ml), len(methylgrapher_ml))




        x = [i+1 for i in range(30)]
        for ps in pipeline_name_short:

            if ps == "bs":
                ml = bismark_ml
                cov = bismark_cov

            if ps == "bc":
                ml = biscuit_ml
                cov = biscuit_cov

            if ps == "bm":
                ml = bwamem_ml
                cov = bwamem_cov

            if ps == "gb":
                ml = gembs_ml
                cov = gembs_cov

            if ps == "mg":
                ml = methylgrapher_ml
                cov = methylgrapher_cov


            cov = numpy.array(cov)
            cov = cov / numpy.sum(cov)

            pipeline_name_full = pipeline_full_name_lookup[ps]
            pipeline_color = pipeline_color_lookup[ps]

            seaborn.kdeplot(ml, ax=ax3a, label=pipeline_name_full, color=pipeline_color, linestyle="-.", alpha=0.9)
            ax3b.plot(x, cov, label=pipeline_name_full, color=pipeline_color, linestyle="-.", alpha=0.9)

            if s == "HG00621" and r == 1:
                seaborn.kdeplot(ml, ax=axes3ab[0], label=pipeline_name_full, color=pipeline_color, linestyle="-.", alpha=0.9)
                axes3ab[1].plot(x, cov, label=pipeline_name_full, color=pipeline_color, linestyle="-.", alpha=0.9)


                axes3ab[0].set_title("CpG Methylation Level Distribution")
                axes3ab[0].set_xlabel("Methylation Level")
                axes3ab[0].set_ylabel("Density")
                axes3ab[0].legend()

                axes3ab[1].set_title("Coverage Distribution")
                axes3ab[1].set_xlabel("Coverage")
                axes3ab[1].set_ylabel("Frequency")
                axes3ab[1].legend()
                fig3ab.savefig("./figures/3AB.pdf", dpi=300, bbox_inches="tight")




        ax3a.set_xlabel("Methylation Level")
        ax3a.set_ylabel("Density")

        ax3b.set_xlabel("Coverage")
        ax3b.set_ylabel("Frequency")


        for ax in [ax3a, ax3b]:
            ax.set_title(f"{s}_{r}")
            ax.legend()


    plt.tight_layout()
    fig3a.savefig("./figures/3A.pdf", dpi=300, bbox_inches="tight")
    fig3b.savefig("./figures/3B.pdf", dpi=300, bbox_inches="tight")





In [ ]:



cpg_count_data_str = f"""
HG00621 1 Cov5 11741845 18071926 18794926 5669852 17858303
HG00621 1 All 27163995 27601865 27569426 26182392 28947783
HG00621 2 Cov5 15910117 20606666 21343261 11662920 20619053
HG00621 2 All 27537025 27728950 27691525 26983109 29179700
HG00741 1 Cov5 21660818 24423465 24844556 18058897 24445013
HG00741 1 All 27898978 27824855 27770452 27335026 29387774
HG00741 2 Cov5 18650025 22053575 22610971 14855493 21929021
HG00741 2 All 27768822 27753136 27706786 27194867 29233908
HG01952 1 Cov5 14305275 21318588 22966103 13016591 22194644
HG01952 1 All 27253305 27698904 27735793 27039254 29330601
HG01952 2 Cov5 19498287 23722900 24479310 17068924 24098535
HG01952 2 All 27764613 27858866 27820443 27316446 29451874
HG01978 1 Cov5 12582107 20737205 22464528 10059453 21538959
HG01978 1 All 26965541 27582194 27614979 26748735 29141587
HG01978 2 Cov5 22278028 25280946 25881356 20306613 25859772
HG01978 2 All 27820561 27817490 27776046 27385860 29452783
HG03516 1 Cov5 23280314 25681414 26062664 20189481 26079132
HG03516 1 All 27907806 27834517 27778623 27396935 29575158
HG03516 2 Cov5 22841755 25624116 26106537 20294108 26153204
HG03516 2 All 27842437 27821775 27775678 27385403 29589049
""".strip().split("\n")




hg38_cpg_count = 29162231
graph_cpg_count = 35769868



cpg_count_data = {}
for s in samples:
    cpg_count_data[s] = {}
    for r in replicates:
        cpg_count_data[s][r] = {}
        for p in pipeline_name_short:
            cpg_count_data[s][r][p] = [0, 0]


for l in cpg_count_data_str:
    l = l.strip().split()
    for i in range(3, 8):
        l[i] = int(l[i])

    s, r, t, bs, bc, bm, gb, mg = l
    r = int(r)

    list_index = 0
    if t == "All":
        list_index = 1

    cpg_count_data[s][r]["bs"][list_index] = bs
    cpg_count_data[s][r]["bc"][list_index] = bc
    cpg_count_data[s][r]["bm"][list_index] = bm
    cpg_count_data[s][r]["gb"][list_index] = gb
    cpg_count_data[s][r]["mg"][list_index] = mg

    # print(s, r, t, bs, bc, bm, gb, mg)



fig3C, axes3C = plt.subplots(2, 1, figsize=(10, 8))


ax3C = axes3C[0]
for p in pipeline_name_short:
    x0_offset = 0.15 * (pipeline_name_short.index(p) - 2)


    x = []
    y1 = []
    y2 = []

    x0 = 0
    for s in samples:
        for r in replicates:
            y1.append(cpg_count_data[s][r][p][0])
            y2.append(cpg_count_data[s][r][p][1])

            x.append(x0 + x0_offset)
            x0 += 1
    
    # print(min(y2), max(y2))
    ax3C.bar(x, y1, width=0.15, label=f"{pipeline_full_name_lookup[p]}", color=pipeline_color_lookup[p])
    ax3C.bar(x, y2, width=0.15, color=pipeline_color_lookup[p], alpha=0.5)



for x0 in range(10):
    ax3C.plot([x0-0.4, x0+0.2], [hg38_cpg_count, hg38_cpg_count],   color="black", alpha=0.5)
    ax3C.plot([x0+0.2, x0+0.35], [graph_cpg_count, graph_cpg_count], color="black", alpha=0.5)



ax3C.text(-0.7, hg38_cpg_count + 1e6, f"hg38: {hg38_cpg_count/1e6:.1f}M", color="black")
ax3C.text(-0.7, graph_cpg_count + 1e6, f"HPRC: {graph_cpg_count/1e6:.1f}M", color="black")

ax3C.set_xticks(numpy.arange(10))
ax3C.set_xticklabels([f"{r}" for s in samples for r in replicates])
ax3C.set_title("CpG Count")
ax3C.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax3C.set_ylim(0, 3.9e7)

ax3C.set_yticks([0, 1e7, 2e7, 3e7])
ax3C.set_yticklabels(["", "10M", "20M", "30M"])
# axes3C.set_ylabel("Count")



axes3C[1].bar([0], [0], width=0.15, label=f"  Cov>5", color="gray")
axes3C[1].bar([0], [0], width=0.15, label=f"0<Cov<=5", color="gray", alpha=0.5)
axes3C[1].legend(loc='upper left', bbox_to_anchor=(1, 1))


fig3C.savefig("./figures/3C.pdf", dpi=300, bbox_inches="tight")








In [ ]:
# Figure 4 Helper Functions


def linear_regression(x, y):
    x = np.array(x).reshape((-1, 1))
    y = np.array(y)

    model = LinearRegression()
    model.fit(x, y)

    k = model.coef_[0]
    b = model.intercept_

    r_sq = model.score(x, y)

    return k, b, r_sq

def pearson(x, y):
    x = np.array(x)
    y = np.array(y)
    r = scipy.stats.pearsonr(x, y)[0]
    return r


def array_fix(hmd):
    dim0 = hmd.shape[0]

    newhmd = np.zeros(hmd.shape)
    for xo in range(dim0):
        for yo in range(dim0):
            xn = dim0-1 - yo
            yn = xo
            newhmd[xn, yn] = hmd[xo, yo]

    return newhmd


def density2d(x, y, bin=40, log=None):
    assert len(x) == len(y)

    shape = (bin+1, bin+1)
    step = 1 / bin
    res = numpy.ones(shape)

    total = 0
    for i in range(len(x)):
        x0 = x[i]
        y0 = y[i]

        # yi = bin - int(x0 / step)
        # xi = int(y0 / step)
        xi = int(x0 / step)
        yi = int(y0 / step)

        # print(x0, xi, y0, yi, step)

        res[xi, yi] += 1

    mm = numpy.max(res)
    total = numpy.sum(res)

    for a in range(bin+1):
        for b in range(bin+1):
            pass
            # print(res[a, b])
            # res[a, b] = math.log2(res[a, b]/total)
            res[a, b] = res[a, b] / mm
            if log != None:
                res[a, b] = math.log2(res[a, b])
            # print(res[a, b])


    return array_fix(res)



def density2dInt(x, y, log=None):
    assert len(x) == len(y)

    lowest = 0
    highest = max(x+y)

    shape = (highest+1, highest+1)
    step = 1
    res = numpy.ones(shape)

    total = 0
    for i in range(len(x)):
        x0 = x[i]
        y0 = y[i]

        # yi = bin - int(x0 / step)
        # xi = int(y0 / step)
        xi = int(x0 / step)
        yi = int(y0 / step)

        # print(x0, xi, y0, yi, step)

        res[xi, yi] += 1

    mm = numpy.max(res)
    total = numpy.sum(res)

    for a in range(highest+1):
        for b in range(highest+1):
            pass
            # print(res[a, b])
            # res[a, b] = math.log2(res[a, b]/total)
            res[a, b] = res[a, b] / mm
            if log != None:
                res[a, b] = math.log2(res[a, b])
            # print(res[a, b])


    return array_fix(res)


def diff_bar_graph_data(delta):

    delta_x = []
    delta_y = []


    diff = [0.1, 0.2, 0.3, 0.4, 0.5]
    for i in range(1, 10):
        diff_range_center = i / 10 - 0.5
        delta_x.append(diff_range_center)
        delta_y.append(0)


    for d in delta:
        if d < -0.45:
            d = -0.45
        if d > 0.44:
            d = 0.44

        for idx, dc in enumerate(delta_x):
            if abs(dc - d) < 0.05:
                delta_y[idx] += 1

    return delta_x, delta_y





In [ ]:
# Figure S4-S7, and 4A, 4B



for pns in pipeline_name_short:
    if pns == "mg":
        continue

    print(pns)
    fig4A, axes4A = plt.subplots(3, 5, figsize=(25, 18))
    plt.subplots_adjust(hspace=0.4)
    for s in samples:
        print("\t", s)

        ml_delta = []

        bs_cpg_cov_r1 = []
        bs_cpg_cov_r2 = []

        bs_cpg_ml_r1 = []
        bs_cpg_ml_r2 = []


        #bsd_r1 = bismark_cpg_data[s][replicates[0]]
        #bsd_r2 = bismark_cpg_data[s][replicates[1]]

        bsd_r1 = methylc_getter(pns, "cpg", s, replicates[0])
        bsd_r2 = methylc_getter(pns, "cpg", s, replicates[1])


        for chrom in bsd_r1:
            if chrom not in bsd_r2:
                continue

            for pos in bsd_r1[chrom]:
                pos = int(pos)

                if pos not in bsd_r2[chrom]:
                    continue

                bsd01 = bsd_r1[chrom][pos]
                bsd02 = bsd_r2[chrom][pos]

                cov1 = bsd01[1]
                cov2 = bsd02[1]
                if cov1 > 30:
                    cov1 = 30
                if cov2 > 30:
                    cov2 = 30
                bs_cpg_cov_r1.append(cov1)
                bs_cpg_cov_r2.append(cov2)

                if bsd01[1] <= min_cov or bsd02[1] <= min_cov:
                    continue

                ml1 = bsd01[0] / bsd01[1]
                ml2 = bsd02[0] / bsd02[1]

                diff = ml1 - ml2
                ml_delta.append(diff)

                bs_cpg_ml_r1.append(ml1)
                bs_cpg_ml_r2.append(ml2)


        ml_k, ml_b, ml_r2 = linear_regression(bs_cpg_ml_r1, bs_cpg_ml_r2)
        ml_r_pearson = pearson(bs_cpg_ml_r1, bs_cpg_ml_r2)

        ml_hmd = density2d(bs_cpg_ml_r1, bs_cpg_ml_r2, bin=60)
        ml_hmd = gaussian_filter(ml_hmd, sigma=2)


        cov_k, cov_b, cov_r2 = linear_regression(bs_cpg_cov_r1, bs_cpg_cov_r2)
        cov_r_pearson = pearson(bs_cpg_cov_r1, bs_cpg_cov_r2)

        cov_hmd = density2dInt(bs_cpg_cov_r1, bs_cpg_cov_r2)





        dim0 = ml_hmd.shape[0]
        bcount = dim0-1
        bcount_half = int(bcount/2)



        axes_xi = samples.index(s)


        ax1 = axes4A[0, axes_xi]
        ax2 = axes4A[1, axes_xi]
        ax3 = axes4A[2, axes_xi]




        # hmd_gb = gaussian_filter(hmd, sigma=2)
        im = ax1.imshow(ml_hmd, cmap="Blues", vmin=0, vmax=0.02) # , vmin=0, vmax=0.02


        ax1.set_title(f"{s}")
        ax1.set_xlabel("Methylation (Rep 1)")
        ax1.set_ylabel("Methylation (Rep 2)")
        ax1.text(32, 56, f"y = {ml_k:0.2f}x + {ml_b:0.2f}\nr = {ml_r_pearson:0.2f}\nN = {len(bs_cpg_ml_r1)/1e6:0.2f} M", fontsize=12, color="black", bbox=dict(facecolor='white', alpha=0.0))
        ax1.set_xticks([0, 0.25*bcount, 0.5*bcount, 0.75*bcount, bcount], [0, 0.25, 0.5, 0.75, 1])
        ax1.set_yticks([bcount, 0.75*bcount, 0.5*bcount, 0.25*bcount, 0], [0, 0.25, 0.5, 0.75, 1])

        ax3.imshow(cov_hmd, cmap="Blues", vmin=0, )
        ax3.text(16, 28, f"y = {cov_k:0.2f}x + {cov_b:0.2f}\nr = {cov_r_pearson:0.2f}\nN = {len(bs_cpg_cov_r1)/1e6:0.2f} M", fontsize=12, color="black", bbox=dict(facecolor='white', alpha=0.0))
        ax3.set_title(f"{s}")

        ax3.set_title(f"Coverage 2D histogram")
        ax3.set_xlabel("Coverage (Rep 1)")
        ax3.set_ylabel("Coverage (Rep 2)")
        ax3.set_xticks([0, 5, 10, 15, 20, 25, 30], [0, 5, 10, 15, 20, 25, 30])
        ax3.set_yticks([0, 5, 10, 15, 20, 25, 30], [30, 25, 20, 15, 10, 5, 0])


        # ax1.colorbar(im, fraction=0.02)




        # seaborn.kdeplot(np.array(ml_delta), bw_method=0.1, ax=ax2)
        delta_x, delta_y = diff_bar_graph_data(ml_delta)
        ax2.bar(delta_x, delta_y, width=0.05)

        # Updates
        ax2.set_title(f"Delta Methylation")
        ax2.set_xlabel("Methylation Diff")
        ax2.set_ylabel("# of CpG (in millions)")

        y0 = max(delta_y) * 1.02
        ax2.plot([-0.43, 0.43], [y0, y0], color='#1f76b4',)
        ax2.plot([0.15, 0.15], [0, y0], linestyle='dashed', color='#1f76b4',)
        ax2.plot([-0.15, -0.15], [0, y0], linestyle='dashed', color='#1f76b4',)

        t = [-0.45, -0.35, -0.25, -0.15, -0.05, 0.05, 0.15, 0.25, 0.35, 0.45]
        tl = [-1, -0.35, -0.25, -0.15, -0.05, 0.05, 0.15, 0.25, 0.35, 1]
        ax2.set_xticks(t, tl, rotation='vertical')

        a = sum(delta_y[:3]) / sum(delta_y) * 100
        b = sum(delta_y[3:6]) / sum(delta_y) * 100
        c = sum(delta_y[6:]) / sum(delta_y) * 100

        ax2.text(-0.35, y0*1.01, f"{a:0.1f} %", fontsize=10, color="#1f76b4")
        ax2.text(-0.05, y0*1.01, f"{b:0.1f} %", fontsize=10, color="#1f76b4")
        ax2.text( 0.25, y0*1.01, f"{c:0.1f} %", fontsize=10, color="#1f76b4")

        concordance = b


        fig4A.savefig(f"./figures/S4_8.{pns}.pdf", bbox_inches="tight")

        assert len(bs_cpg_ml_r1) == len(bs_cpg_ml_r2)
        print(f"\t\tY= {ml_k:0.4f}X + {ml_b:0.4f}, R^2 = {ml_r2:0.4f}, Pearson = {ml_r_pearson:0.4f}, N={len(bs_cpg_ml_r1)/1e6:0.1f}M, concordance = {concordance:0.2f}%")
        print(f"\t\tY= {cov_k:0.4f}X + {cov_b:0.4f}, R^2 = {cov_r2:0.4f}, Pearson = {cov_r_pearson:0.4f} N={len(bs_cpg_cov_r1)/1e6:0.1f}M")





In [ ]:
# Figure S8, 4C, 4D




# Make subplot space larger
fig4B, axes4B = plt.subplots(3, 5, figsize=(25, 18))
plt.subplots_adjust(hspace=0.4)

print("mg")
for s in samples:
    print("\t"+s)


    ml_delta = []
    mg_cpg_ml_r1 = []
    mg_cpg_ml_r2 = []

    mg_cpg_cov_r1 = []
    mg_cpg_cov_r2 = []

    mgd_r1 = methylgrapher_cpg_data[s][replicates[0]]
    mgd_r2 = methylgrapher_cpg_data[s][replicates[1]]

    for cpg_id in mgd_r1:
        if cpg_id not in mgd_r2:
            continue


        mgd01 = mgd_r1[cpg_id]
        mgd02 = mgd_r2[cpg_id]

        cov1 = mgd01[1]
        cov2 = mgd02[1]
        if cov1 > 30:
            cov1 = 30
        if cov2 > 30:
            cov2 = 30
        mg_cpg_cov_r1.append(cov1)
        mg_cpg_cov_r2.append(cov2)

        if mgd01[1] <= min_cov or mgd02[1] <= min_cov:
            continue

        ml1 = mgd01[0] / mgd01[1]
        ml2 = mgd02[0] / mgd02[1]

        diff = ml1 - ml2
        ml_delta.append(diff)

        mg_cpg_ml_r1.append(ml1)
        mg_cpg_ml_r2.append(ml2)



    ml_k, ml_b, ml_r2 = linear_regression(mg_cpg_ml_r1, mg_cpg_ml_r2)
    ml_r_pearson = pearson(mg_cpg_ml_r1, mg_cpg_ml_r2)

    ml_hmd = density2d(mg_cpg_ml_r1, mg_cpg_ml_r2, bin=60)


    cov_k, cov_b, cov_r2 = linear_regression(mg_cpg_cov_r1, mg_cpg_cov_r2)
    cov_r_pearson = pearson(mg_cpg_cov_r1, mg_cpg_cov_r2)

    cov_hmd = density2dInt(mg_cpg_cov_r1, mg_cpg_cov_r2)





    dim0 = ml_hmd.shape[0]
    bcount = dim0-1
    bcount_half = int(bcount/2)



    axes_xi = samples.index(s)


    ax1 = axes4B[0, axes_xi]
    ax2 = axes4B[1, axes_xi]
    ax3 = axes4B[2, axes_xi]




    ml_hmd = gaussian_filter(ml_hmd, sigma=2)
    im = ax1.imshow(ml_hmd, cmap="Blues", vmin=0, vmax=0.02) # , vmin=0, vmax=0.02


    ax1.set_title(f"{s}")
    ax1.set_xlabel("Methylation (Rep 1)")
    ax1.set_ylabel("Methylation (Rep 2)")
    ax1.text(32, 56, f"y = {ml_k:0.2f}x + {ml_b:0.2f}\nr = {ml_r_pearson:0.2f}\nN = {len(mg_cpg_ml_r1)/1e6:0.2f} M", fontsize=12, color="black", bbox=dict(facecolor='white', alpha=0.0))
    ax1.set_xticks([0, 0.25*bcount, 0.5*bcount, 0.75*bcount, bcount], [0, 0.25, 0.5, 0.75, 1])
    ax1.set_yticks([bcount, 0.75*bcount, 0.5*bcount, 0.25*bcount, 0], [0, 0.25, 0.5, 0.75, 1])

    ax3.imshow(cov_hmd, cmap="Blues", vmin=0, )
    ax3.text(16, 28, f"y = {cov_k:0.2f}x + {cov_b:0.2f}\nr = {cov_r_pearson:0.2f}\nN = {len(mg_cpg_cov_r1)/1e6:0.2f} M", fontsize=12, color="black", bbox=dict(facecolor='white', alpha=0.0))
    ax3.set_title(f"{s}")

    ax3.set_title(f"Coverage 2D histogram")
    ax3.set_xlabel("Coverage (Rep 1)")
    ax3.set_ylabel("Coverage (Rep 2)")
    ax3.set_xticks([0, 5, 10, 15, 20, 25, 30], [0, 5, 10, 15, 20, 25, 30])
    ax3.set_yticks([0, 5, 10, 15, 20, 25, 30], [30, 25, 20, 15, 10, 5, 0])


    # ax1.colorbar(im, fraction=0.02)




    # seaborn.kdeplot(np.array(ml_delta), bw_method=0.1, ax=ax2)
    delta_x, delta_y = diff_bar_graph_data(ml_delta)
    ax2.bar(delta_x, delta_y, width=0.05)

    # Updates
    ax2.set_title(f"Delta Methylation")
    ax2.set_xlabel("Methylation Diff")
    ax2.set_ylabel("# of CpG (in millions)")

    y0 = max(delta_y) * 1.02
    ax2.plot([-0.43, 0.43], [y0, y0], color='#1f76b4',)
    ax2.plot([0.15, 0.15], [0, y0], linestyle='dashed', color='#1f76b4',)
    ax2.plot([-0.15, -0.15], [0, y0], linestyle='dashed', color='#1f76b4',)

    t = [-0.45, -0.35, -0.25, -0.15, -0.05, 0.05, 0.15, 0.25, 0.35, 0.45]
    tl = [-1, -0.35, -0.25, -0.15, -0.05, 0.05, 0.15, 0.25, 0.35, 1]
    ax2.set_xticks(t, tl, rotation='vertical')

    a = sum(delta_y[:3]) / sum(delta_y) * 100
    b = sum(delta_y[3:6]) / sum(delta_y) * 100
    c = sum(delta_y[6:]) / sum(delta_y) * 100

    ax2.text(-0.35, y0*1.01, f"{a:0.1f} %", fontsize=10, color="#1f76b4")
    ax2.text(-0.05, y0*1.01, f"{b:0.1f} %", fontsize=10, color="#1f76b4")
    ax2.text( 0.25, y0*1.01, f"{c:0.1f} %", fontsize=10, color="#1f76b4")

    concordance = b



    fig4B.savefig(f"./figures/S4_8.mg.pdf", bbox_inches="tight")

    assert len(mg_cpg_cov_r1) == len(mg_cpg_cov_r2)
    print(f"\t\tY= {ml_k:0.4f}X + {ml_b:0.4f}, R^2 = {ml_r2:0.4f}, Pearson = {ml_r_pearson:0.4f}, N={len(mg_cpg_ml_r1)/1e6:0.1f}M, concordance = {concordance:0.2f}%")
    print(f"\t\tY= {cov_k:0.4f}X + {cov_b:0.4f}, R^2 = {cov_r2:0.4f}, Pearson = {cov_r_pearson:0.4f}, N={len(mg_cpg_cov_r2)/1e6:0.1f}M")
    print()






In [ ]:
# Figure S9_13, 4E, 4F


log_fh = open("figures/S9_13_4E_4F.log", "w")

for pns in pipeline_name_short:
    if pns == "mg":
        continue
    
    print(pns)
    fig4C, axes4C = plt.subplots(6, 5, figsize=(25, 40))
    plt.subplots_adjust(hspace=0.4)
    for s in samples:
        for r in replicates:


            print("\t", s, r)

            ml_delta = []

            bs_cpg_ml = []
            mg_cpg_ml = []

            bs_cpg_cov = []
            mg_cpg_cov = []

            # "bsd" used to mean bismark data, now it means linear method data
            # bsd = bismark_cpg_data[s][r]
            bsd = methylc_getter(pns, "cpg", s, r)
            mg2ld = methylgrapher_lifted_cytosine_data[s][r]

            found = 0
            for chrom in bsd:
                if chrom not in mg2ld:
                    continue

                for pos in bsd[chrom]:
                    pos = int(pos)

                    bsd0 = bsd[chrom][pos]
                    ml_bs = bsd0[0] / bsd0[1]


                    mgd01 = mg2ld[chrom].get(pos, (0, 0))
                    mgd02 = mg2ld[chrom].get(pos+1, (0, 0))

                    mg_met = mgd01[0] + mgd02[0]
                    mg_cov = mgd01[1] + mgd02[1]

                    if mg_cov == 0:
                        continue

                    bs_cov0 = bsd0[1]
                    mg_cov0 = mg_cov

                    if bs_cov0 > 30:
                        bs_cov0 = 30
                    if mg_cov0 > 30:
                        mg_cov0 = 30

                    bs_cpg_cov.append(bs_cov0)
                    mg_cpg_cov.append(mg_cov0)


                    if bsd0[1] <= min_cov:
                        continue

                    if mg_cov <= min_cov:
                        continue

                    ml_mg = mg_met / mg_cov

                    # print(ml_bs, f"{mg_met}/{mg_cov}", mgd01, mgd02)

                    diff = ml_bs - ml_mg
                    ml_delta.append(diff)

                    bs_cpg_ml.append(ml_bs)
                    mg_cpg_ml.append(ml_mg)






            ml_k, ml_b, ml_r2 = linear_regression(bs_cpg_ml, mg_cpg_ml)
            ml_r_pearson = pearson(bs_cpg_ml, mg_cpg_ml)

            ml_hmd = density2d(bs_cpg_ml, mg_cpg_ml, bin=60)


            cov_k, cov_b, cov_r2 = linear_regression(bs_cpg_cov, mg_cpg_cov)
            cov_r_pearson = pearson(bs_cpg_cov, mg_cpg_cov)

            cov_hmd = density2dInt(bs_cpg_cov, mg_cpg_cov)



            dim0 = ml_hmd.shape[0]
            bcount = dim0-1
            bcount_half = int(bcount/2)




            axes_xi = samples.index(s)
            axes_yi = replicates.index(r)
            if axes_yi == 1:
                axes_yi += 2

            ax1 = axes4C[axes_yi, axes_xi]
            ax2 = axes4C[axes_yi+1, axes_xi]
            ax3 = axes4C[axes_yi+2, axes_xi]


            ml_hmd = gaussian_filter(ml_hmd, sigma=2)
            im = ax1.imshow(ml_hmd, cmap="Blues", vmin=0, vmax=0.02) # , vmin=0, vmax=0.02


            ax1.set_title(f"{s}_{r}")
            ax1.set_xlabel(f"Methylation ({pipeline_full_name_lookup[pns]})")
            ax1.set_ylabel("Methylation (methylGrapher)")
            ax1.text(32, 56, f"y = {ml_k:0.2f}x + {ml_b:0.2f}\nr = {ml_r_pearson:0.2f}\nN = {len(mg_cpg_ml)/1e6:0.2f} M", fontsize=12, color="black", bbox=dict(facecolor='white', alpha=0.0))
            ax1.set_xticks([0, 0.25*bcount, 0.5*bcount, 0.75*bcount, bcount], [0, 0.25, 0.5, 0.75, 1])
            ax1.set_yticks([bcount, 0.75*bcount, 0.5*bcount, 0.25*bcount, 0], [0, 0.25, 0.5, 0.75, 1])

            ax3.imshow(cov_hmd, cmap="Blues", vmin=0, )
            ax3.text(16, 28, f"y = {cov_k:0.2f}x + {cov_b:0.2f}\nr = {cov_r_pearson:0.2f}\nN = {len(mg_cpg_cov)/1e6:0.2f} M", fontsize=12, color="black", bbox=dict(facecolor='white', alpha=0.0))
            ax3.set_title(f"{s}")

            ax3.set_title(f"Coverage 2D histogram")
            ax3.set_xlabel(f"Coverage ({pipeline_full_name_lookup[pns]})")
            ax3.set_ylabel(f"Coverage (methylGrapher)")
            ax3.set_xticks([0, 5, 10, 15, 20, 25, 30], [0, 5, 10, 15, 20, 25, 30])
            ax3.set_yticks([0, 5, 10, 15, 20, 25, 30], [30, 25, 20, 15, 10, 5, 0])


            # ax1.colorbar(im, fraction=0.02)




            # seaborn.kdeplot(np.array(ml_delta), bw_method=0.1, ax=ax2)
            delta_x, delta_y = diff_bar_graph_data(ml_delta)
            ax2.bar(delta_x, delta_y, width=0.05)

            # Updates
            ax2.set_title(f"Delta Methylation")
            ax2.set_xlabel("Methylation Diff")
            ax2.set_ylabel("# of CpG (in millions)")

            y0 = max(delta_y) * 1.02
            ax2.plot([-0.43, 0.43], [y0, y0], color='#1f76b4',)
            ax2.plot([0.15, 0.15], [0, y0], linestyle='dashed', color='#1f76b4',)
            ax2.plot([-0.15, -0.15], [0, y0], linestyle='dashed', color='#1f76b4',)

            t = [-0.45, -0.35, -0.25, -0.15, -0.05, 0.05, 0.15, 0.25, 0.35, 0.45]
            tl = [-1, -0.35, -0.25, -0.15, -0.05, 0.05, 0.15, 0.25, 0.35, 1]
            ax2.set_xticks(t, tl, rotation='vertical')

            a = sum(delta_y[:3]) / sum(delta_y) * 100
            b = sum(delta_y[3:6]) / sum(delta_y) * 100
            c = sum(delta_y[6:]) / sum(delta_y) * 100

            ax2.text(-0.35, y0*1.01, f"{a:0.1f} %", fontsize=10, color="#1f76b4")
            ax2.text(-0.05, y0*1.01, f"{b:0.1f} %", fontsize=10, color="#1f76b4")
            ax2.text( 0.25, y0*1.01, f"{c:0.1f} %", fontsize=10, color="#1f76b4")

            concordance = b




            fig4C.savefig(f"./figures/S9_13.{pns}.pdf", bbox_inches="tight")

            assert len(mg_cpg_cov) == len(mg_cpg_cov)
            s1 = f"\t\tY= {ml_k:0.4f}X + {ml_b:0.4f}, R^2 = {ml_r2:0.4f}, Pearson = {ml_r_pearson:0.4f}, N={len(mg_cpg_ml)/1e6:0.1f}M, concordance = {concordance:0.2f}%"
            s2 = f"\t\tY= {cov_k:0.4f}X + {cov_b:0.4f}, R^2 = {cov_r2:0.4f}, Pearson = {cov_r_pearson:0.4f}, N={len(mg_cpg_cov)/1e6:0.1f}M"

            print(s1)
            print(s2)

            log_fh.write(pns + "\n")
            log_fh.write(f"\t{s}\t{r}\n")
            log_fh.write(s1 + "\n")
            log_fh.write(s2 + "\n")
            log_fh.flush()


log_fh.close()







In [ ]:


replicate_comparison_str = f"""
bs
	 HG00621
		Y= 0.8769X + 0.0938, R^2 = 0.7724, Pearson = 0.8788, N=8.5M, concordance = 72.17%
		Y= 0.4622X + 4.1579, R^2 = 0.1619, Pearson = 0.4024, N=26.7M
	 HG00741
		Y= 0.8889X + 0.0691, R^2 = 0.7742, Pearson = 0.8799, N=16.1M, concordance = 68.47%
		Y= 0.3640X + 4.1688, R^2 = 0.1672, Pearson = 0.4089, N=27.5M
	 HG01952
		Y= 0.8577X + 0.0868, R^2 = 0.7520, Pearson = 0.8672, N=12.1M, concordance = 66.48%
		Y= 0.5292X + 4.8450, R^2 = 0.2088, Pearson = 0.4569, N=26.9M
	 HG01978
		Y= 0.8629X + 0.0866, R^2 = 0.7707, Pearson = 0.8779, N=11.7M, concordance = 67.46%
		Y= 0.6521X + 6.1592, R^2 = 0.2152, Pearson = 0.4639, N=26.7M
	 HG03516
		Y= 0.8963X + 0.0635, R^2 = 0.7989, Pearson = 0.8938, N=20.6M, concordance = 71.07%
		Y= 0.5026X + 4.9468, R^2 = 0.2424, Pearson = 0.4923, N=27.6M
bc
	 HG00621
		Y= 0.8500X + 0.1134, R^2 = 0.7246, Pearson = 0.8512, N=15.0M, concordance = 65.97%
		Y= 0.4307X + 5.2311, R^2 = 0.1543, Pearson = 0.3928, N=27.4M
	 HG00741
		Y= 0.8763X + 0.0733, R^2 = 0.7483, Pearson = 0.8650, N=20.5M, concordance = 64.28%
		Y= 0.3437X + 5.1144, R^2 = 0.1521, Pearson = 0.3901, N=27.6M
	 HG01952
		Y= 0.8493X + 0.0904, R^2 = 0.7341, Pearson = 0.8568, N=19.5M, concordance = 63.47%
		Y= 0.4473X + 6.4740, R^2 = 0.1743, Pearson = 0.4175, N=27.5M
	 HG01978
		Y= 0.8562X + 0.0931, R^2 = 0.7517, Pearson = 0.8670, N=19.8M, concordance = 64.67%
		Y= 0.5213X + 7.9744, R^2 = 0.1775, Pearson = 0.4213, N=27.4M
	 HG03516
		Y= 0.8919X + 0.0639, R^2 = 0.7925, Pearson = 0.8902, N=24.5M, concordance = 69.12%
		Y= 0.4767X + 6.9044, R^2 = 0.2135, Pearson = 0.4620, N=27.7M
bm
	 HG00621
		Y= 0.8523X + 0.1110, R^2 = 0.7287, Pearson = 0.8536, N=15.9M, concordance = 66.01%
		Y= 0.4136X + 5.5640, R^2 = 0.1423, Pearson = 0.3772, N=27.4M
	 HG00741
		Y= 0.8784X + 0.0716, R^2 = 0.7518, Pearson = 0.8671, N=21.2M, concordance = 64.47%
		Y= 0.3316X + 5.3908, R^2 = 0.1406, Pearson = 0.3749, N=27.6M
	 HG01952
		Y= 0.8556X + 0.0867, R^2 = 0.7428, Pearson = 0.8619, N=21.4M, concordance = 64.26%
		Y= 0.4410X + 6.6710, R^2 = 0.1757, Pearson = 0.4192, N=27.6M
	 HG01978
		Y= 0.8608X + 0.0905, R^2 = 0.7594, Pearson = 0.8714, N=21.8M, concordance = 65.44%
		Y= 0.5066X + 8.3419, R^2 = 0.1765, Pearson = 0.4201, N=27.5M
	 HG03516
		Y= 0.8949X + 0.0634, R^2 = 0.7983, Pearson = 0.8935, N=25.2M, concordance = 69.67%
		Y= 0.4550X + 7.6022, R^2 = 0.1937, Pearson = 0.4401, N=27.6M
gb
	 HG00621
		Y= 0.8782X + 0.0917, R^2 = 0.7807, Pearson = 0.8836, N=3.3M, concordance = 70.81%
		Y= 0.3919X + 3.8824, R^2 = 0.1013, Pearson = 0.3182, N=25.6M
	 HG00741
		Y= 0.8843X + 0.0710, R^2 = 0.7709, Pearson = 0.8780, N=11.1M, concordance = 67.33%
		Y= 0.2973X + 4.0320, R^2 = 0.1068, Pearson = 0.3268, N=26.9M
	 HG01952
		Y= 0.8680X + 0.0797, R^2 = 0.7613, Pearson = 0.8725, N=9.7M, concordance = 66.62%
		Y= 0.4187X + 4.5256, R^2 = 0.1407, Pearson = 0.3751, N=26.7M
	 HG01978
		Y= 0.8729X + 0.0794, R^2 = 0.7819, Pearson = 0.8842, N=8.7M, concordance = 67.62%
		Y= 0.5130X + 5.6525, R^2 = 0.1412, Pearson = 0.3757, N=26.5M
	 HG03516
		Y= 0.8899X + 0.0674, R^2 = 0.7919, Pearson = 0.8899, N=16.4M, concordance = 69.88%
		Y= 0.4163X + 4.7974, R^2 = 0.1657, Pearson = 0.4071, N=27.1M
mg
	HG00621
		Y= 0.8846X + 0.0883, R^2 = 0.7840, Pearson = 0.8854, N=14.8M, concordance = 73.60%
		Y= 0.5048X + 4.6176, R^2 = 0.2104, Pearson = 0.4587, N=28.4M

	HG00741
		Y= 0.8980X + 0.0636, R^2 = 0.7883, Pearson = 0.8879, N=20.2M, concordance = 70.03%
		Y= 0.4216X + 4.1696, R^2 = 0.2306, Pearson = 0.4802, N=28.8M

	HG01952
		Y= 0.8756X + 0.0763, R^2 = 0.7760, Pearson = 0.8809, N=20.2M, concordance = 69.16%
		Y= 0.5421X + 5.4063, R^2 = 0.2563, Pearson = 0.5062, N=28.9M

	HG01978
		Y= 0.8784X + 0.0790, R^2 = 0.7898, Pearson = 0.8887, N=20.7M, concordance = 69.99%
		Y= 0.6327X + 6.8658, R^2 = 0.2611, Pearson = 0.5110, N=28.8M

	HG03516
		Y= 0.9077X + 0.0573, R^2 = 0.8215, Pearson = 0.9064, N=24.7M, concordance = 73.57%
		Y= 0.5645X + 5.7100, R^2 = 0.3003, Pearson = 0.5480, N=29.2M

""".strip()


method_comparison_str = """
bs
	HG00621	1
		Y= 0.9692X + 0.0229, R^2 = 0.9658, Pearson = 0.9827, N=10.9M, concordance = 96.30%
		Y= 0.9242X + 2.2163, R^2 = 0.5941, Pearson = 0.7708, N=26.3M
	HG00621	2
		Y= 0.9713X + 0.0219, R^2 = 0.9677, Pearson = 0.9837, N=14.9M, concordance = 96.90%
		Y= 0.9126X + 2.2870, R^2 = 0.6282, Pearson = 0.7926, N=26.7M
	HG00741	1
		Y= 0.9743X + 0.0165, R^2 = 0.9697, Pearson = 0.9847, N=20.5M, concordance = 96.68%
		Y= 0.8998X + 2.5798, R^2 = 0.6266, Pearson = 0.7916, N=27.1M
	HG00741	2
		Y= 0.9769X + 0.0148, R^2 = 0.9734, Pearson = 0.9866, N=17.5M, concordance = 97.07%
		Y= 0.8948X + 2.0800, R^2 = 0.6387, Pearson = 0.7992, N=27.0M
	HG01952	1
		Y= 0.9539X + 0.0309, R^2 = 0.9518, Pearson = 0.9756, N=13.6M, concordance = 93.33%
		Y= 0.9322X + 3.5325, R^2 = 0.5535, Pearson = 0.7440, N=26.5M
	HG01952	2
		Y= 0.9649X + 0.0240, R^2 = 0.9620, Pearson = 0.9808, N=18.5M, concordance = 95.30%
		Y= 0.8910X + 3.3579, R^2 = 0.5908, Pearson = 0.7686, N=27.0M
	HG01978	1
		Y= 0.9548X + 0.0293, R^2 = 0.9525, Pearson = 0.9760, N=11.9M, concordance = 92.77%
		Y= 0.9406X + 3.5877, R^2 = 0.5318, Pearson = 0.7292, N=26.2M
	HG01978	2
		Y= 0.9666X + 0.0233, R^2 = 0.9633, Pearson = 0.9815, N=21.3M, concordance = 95.41%
		Y= 0.8749X + 4.0541, R^2 = 0.5921, Pearson = 0.7695, N=27.1M
	HG03516	1
		Y= 0.9701X + 0.0205, R^2 = 0.9653, Pearson = 0.9825, N=22.2M, concordance = 95.86%
		Y= 0.8926X + 3.6391, R^2 = 0.6038, Pearson = 0.7770, N=27.1M
	HG03516	2
		Y= 0.9673X + 0.0231, R^2 = 0.9624, Pearson = 0.9810, N=21.9M, concordance = 95.30%
		Y= 0.8909X + 4.0830, R^2 = 0.5896, Pearson = 0.7678, N=27.1M
bc
	HG00621	1
		Y= 0.9949X + 0.0053, R^2 = 0.9860, Pearson = 0.9930, N=16.5M, concordance = 98.75%
		Y= 0.8935X + 0.6233, R^2 = 0.7770, Pearson = 0.8815, N=26.8M
	HG00621	2
		Y= 0.9930X + 0.0070, R^2 = 0.9852, Pearson = 0.9926, N=19.2M, concordance = 98.74%
		Y= 0.9018X + 0.7093, R^2 = 0.7828, Pearson = 0.8847, N=27.0M
	HG00741	1
		Y= 0.9952X + 0.0047, R^2 = 0.9860, Pearson = 0.9930, N=23.1M, concordance = 98.59%
		Y= 0.9066X + 0.7746, R^2 = 0.7696, Pearson = 0.8773, N=27.2M
	HG00741	2
		Y= 0.9955X + 0.0041, R^2 = 0.9879, Pearson = 0.9939, N=20.6M, concordance = 98.78%
		Y= 0.9044X + 0.6771, R^2 = 0.7750, Pearson = 0.8803, N=27.1M
	HG01952	1
		Y= 0.9898X + 0.0100, R^2 = 0.9792, Pearson = 0.9896, N=19.9M, concordance = 97.63%
		Y= 0.8611X + 1.4749, R^2 = 0.7034, Pearson = 0.8387, N=27.0M
	HG01952	2
		Y= 0.9926X + 0.0084, R^2 = 0.9825, Pearson = 0.9912, N=22.4M, concordance = 98.11%
		Y= 0.8971X + 1.0633, R^2 = 0.7656, Pearson = 0.8750, N=27.2M
	HG01978	1
		Y= 0.9911X + 0.0091, R^2 = 0.9800, Pearson = 0.9900, N=19.2M, concordance = 97.61%
		Y= 0.8659X + 1.3575, R^2 = 0.7083, Pearson = 0.8416, N=26.9M
	HG01978	2
		Y= 0.9915X + 0.0096, R^2 = 0.9813, Pearson = 0.9906, N=24.1M, concordance = 97.92%
		Y= 0.8977X + 1.3548, R^2 = 0.7587, Pearson = 0.8710, N=27.2M
	HG03516	1
		Y= 0.9942X + 0.0065, R^2 = 0.9829, Pearson = 0.9914, N=24.5M, concordance = 98.15%
		Y= 0.9047X + 1.0206, R^2 = 0.7671, Pearson = 0.8758, N=27.2M
	HG03516	2
		Y= 0.9928X + 0.0089, R^2 = 0.9811, Pearson = 0.9905, N=24.5M, concordance = 97.91%
		Y= 0.9041X + 1.1808, R^2 = 0.7667, Pearson = 0.8756, N=27.2M
bm
	HG00621	1
		Y= 0.9969X + 0.0035, R^2 = 0.9864, Pearson = 0.9932, N=16.9M, concordance = 98.87%
		Y= 0.8970X + 0.4076, R^2 = 0.7908, Pearson = 0.8892, N=26.9M
	HG00621	2
		Y= 0.9954X + 0.0048, R^2 = 0.9861, Pearson = 0.9930, N=19.6M, concordance = 98.86%
		Y= 0.9024X + 0.4551, R^2 = 0.7929, Pearson = 0.8905, N=27.0M
	HG00741	1
		Y= 0.9972X + 0.0033, R^2 = 0.9870, Pearson = 0.9935, N=23.3M, concordance = 98.74%
		Y= 0.9087X + 0.5221, R^2 = 0.7721, Pearson = 0.8787, N=27.2M
	HG00741	2
		Y= 0.9973X + 0.0029, R^2 = 0.9888, Pearson = 0.9944, N=20.9M, concordance = 98.90%
		Y= 0.9057X + 0.4738, R^2 = 0.7793, Pearson = 0.8828, N=27.1M
	HG01952	1
		Y= 0.9950X + 0.0043, R^2 = 0.9834, Pearson = 0.9917, N=21.1M, concordance = 98.40%
		Y= 0.8939X + 0.4717, R^2 = 0.7829, Pearson = 0.8848, N=27.1M
	HG01952	2
		Y= 0.9955X + 0.0038, R^2 = 0.9853, Pearson = 0.9926, N=22.9M, concordance = 98.56%
		Y= 0.9091X + 0.5112, R^2 = 0.7870, Pearson = 0.8871, N=27.2M
	HG01978	1
		Y= 0.9952X + 0.0038, R^2 = 0.9836, Pearson = 0.9918, N=20.4M, concordance = 98.34%
		Y= 0.8951X + 0.4278, R^2 = 0.7834, Pearson = 0.8851, N=27.0M
	HG01978	2
		Y= 0.9949X + 0.0044, R^2 = 0.9846, Pearson = 0.9923, N=24.6M, concordance = 98.44%
		Y= 0.9156X + 0.5843, R^2 = 0.7824, Pearson = 0.8845, N=27.2M
	HG03516	1
		Y= 0.9967X + 0.0040, R^2 = 0.9846, Pearson = 0.9923, N=24.7M, concordance = 98.40%
		Y= 0.9096X + 0.5993, R^2 = 0.7672, Pearson = 0.8759, N=27.2M
	HG03516	2
		Y= 0.9960X + 0.0046, R^2 = 0.9837, Pearson = 0.9918, N=24.8M, concordance = 98.29%
		Y= 0.9091X + 0.5953, R^2 = 0.7695, Pearson = 0.8772, N=27.2M
gb
	HG00621	1
		Y= 0.9457X + 0.0432, R^2 = 0.9429, Pearson = 0.9710, N=5.4M, concordance = 92.04%
		Y= 1.0821X + 3.0529, R^2 = 0.4831, Pearson = 0.6950, N=25.6M
	HG00621	2
		Y= 0.9533X + 0.0376, R^2 = 0.9492, Pearson = 0.9743, N=11.1M, concordance = 94.02%
		Y= 1.0663X + 2.6510, R^2 = 0.5814, Pearson = 0.7625, N=26.5M
	HG00741	1
		Y= 0.9536X + 0.0323, R^2 = 0.9510, Pearson = 0.9752, N=17.4M, concordance = 93.22%
		Y= 1.0349X + 3.1474, R^2 = 0.5855, Pearson = 0.7652, N=26.9M
	HG00741	2
		Y= 0.9598X + 0.0278, R^2 = 0.9568, Pearson = 0.9782, N=14.2M, concordance = 93.98%
		Y= 1.0116X + 2.5055, R^2 = 0.6025, Pearson = 0.7762, N=26.7M
	HG01952	1
		Y= 0.9483X + 0.0367, R^2 = 0.9423, Pearson = 0.9707, N=12.5M, concordance = 91.27%
		Y= 1.1027X + 3.0240, R^2 = 0.5764, Pearson = 0.7592, N=26.5M
	HG01952	2
		Y= 0.9524X + 0.0339, R^2 = 0.9484, Pearson = 0.9738, N=16.4M, concordance = 92.66%
		Y= 1.0763X + 3.1812, R^2 = 0.5989, Pearson = 0.7739, N=26.8M
	HG01978	1
		Y= 0.9466X + 0.0367, R^2 = 0.9414, Pearson = 0.9702, N=9.7M, concordance = 90.02%
		Y= 1.1309X + 3.3252, R^2 = 0.5326, Pearson = 0.7298, N=26.2M
	HG01978	2
		Y= 0.9547X + 0.0331, R^2 = 0.9490, Pearson = 0.9742, N=19.7M, concordance = 92.73%
		Y= 1.0885X + 3.7784, R^2 = 0.5985, Pearson = 0.7736, N=26.9M
	HG03516	1
		Y= 0.9501X + 0.0361, R^2 = 0.9463, Pearson = 0.9728, N=19.6M, concordance = 92.22%
		Y= 1.0780X + 4.1794, R^2 = 0.5630, Pearson = 0.7503, N=26.9M
	HG03516	2
		Y= 0.9512X + 0.0358, R^2 = 0.9460, Pearson = 0.9726, N=19.7M, concordance = 92.12%
		Y= 1.0916X + 4.2599, R^2 = 0.5667, Pearson = 0.7528, N=26.9M""".strip()

with open("figures/S9_13_4E_4F.log") as log_fh:
    method_comparison_str = log_fh.read().strip()


replicate_comparison_data_pearson = {}
replicate_comparison_data_concordance = {}

sample = None
pns = None
for l in replicate_comparison_str.split("\n"):
    l = l.strip()
    if l == "":
        continue

    ll = l.split(",")
    for i in range(len(ll)):
        ll[i] = ll[i].strip()

    if len(ll) == 1:
        if ll[0] in ["bs", "bc", "bm", "gb", "mg"]:
            pns = ll[0]
        elif ll[0].startswith("HG0"):
            sample = ll[0]
        else:
            raise RuntimeError

        continue

    # print(ll)
    if len(ll) < 5:
        continue
    if ll[4] == "":
        continue


    pearson_sc = float(ll[2].split("=")[1])
    concordance = float(ll[4].split("=")[1][:-1]) / 100
    # print(pns, sample, pearson_sc, concordance)

    if pns not in replicate_comparison_data_pearson:
        replicate_comparison_data_pearson[pns] = {}
        replicate_comparison_data_concordance[pns] = {}

    # print(pns, sample, pearson_sc, concordance)
    replicate_comparison_data_pearson[pns][sample] = pearson_sc
    replicate_comparison_data_concordance[pns][sample] = concordance



method_comparison_data_pearson = {}
method_comparison_data_concordance = {}

sample = None
pns = None
for l in method_comparison_str.split("\n"):
    l = l.strip()
    if l == "":
        continue

    ll = l.split(",")
    for i in range(len(ll)):
        ll[i] = ll[i].strip()

    if len(ll) == 1:
        lll = ll[0].strip().split()

        if len(lll) == 1:
            assert lll[0] in ["bs", "bc", "bm", "gb", "mg"]
            pns = lll[0]
            continue

        if lll[0].startswith("HG0"):
            sample = lll[0]
            replicate = int(lll[1])
            continue


        sample = ll[0]
        replicate = int(ll[1])
        continue

    if len(ll) < 5:
        continue
    if ll[4] == "":
        continue

    # print(pns, sample, replicate, ll)
    pearson_sc = float(ll[2].split("=")[1])
    concordance = float(ll[4].split("=")[1][:-1]) / 100

    if pns not in method_comparison_data_pearson:
        method_comparison_data_pearson[pns] = {}
        method_comparison_data_concordance[pns] = {}

    method_comparison_data_pearson[pns][(sample, replicate)] = pearson_sc
    method_comparison_data_concordance[pns][(sample, replicate)] = concordance


for pns in method_comparison_data_pearson:
    # print(pns)
    for s in method_comparison_data_pearson[pns]:
        # print(s, method_comparison_data_pearson[pns][s], method_comparison_data_concordance[pns][s])
        pass
    # print()






fig4XXX, axes4XXX = plt.subplots(5, 1, figsize=(8, 25))
# h space
fig4XXX.subplots_adjust(hspace=0.5)


ax1 = axes4XXX[0]
ax2 = axes4XXX[1]
ax3 = axes4XXX[2]
ax4 = axes4XXX[3]
ax5 = axes4XXX[4]

# 5 methods * 5 samples
hmd_replicate_pearson = numpy.zeros((5, 5))
hmd_replicate_concordance = numpy.zeros((5, 5))


for i, pns in enumerate(pipeline_name_short):
    for j, sample in enumerate(samples):
        hmd_replicate_pearson[i, j] = replicate_comparison_data_pearson[pns][sample]
        hmd_replicate_concordance[i, j] = replicate_comparison_data_concordance[pns][sample]



ax1.imshow(hmd_replicate_pearson, cmap="Reds", vmin=0.6, vmax=1)
ax1.set_title("Replicate Pearson Correlation")
ax1.set_xticks(numpy.arange(len(samples)))
ax1.set_xticklabels(samples, rotation=45)
ax1.set_yticks(numpy.arange(len(pipeline_name_short)))
ax1.set_yticklabels(pipeline_name_full)
ax1.grid(False)

ax2.imshow(hmd_replicate_concordance, cmap="Reds", vmin=0.6, vmax=1)
ax2.set_title("Replicate Concordance")
ax2.set_xticks(numpy.arange(len(samples)))
ax2.set_xticklabels(samples, rotation=45)
ax2.set_yticks(numpy.arange(len(pipeline_name_short)))
ax2.set_yticklabels(pipeline_name_full)
ax2.grid(False)

for i in range(len(pipeline_name_short)):
    for j in range(len(samples)):
        ax1.text(j, i, f"{hmd_replicate_pearson[i, j]:.2f}", ha="center", va="center", color="white")
        ax2.text(j, i, f"{hmd_replicate_concordance[i, j]:.2f}", ha="center", va="center", color="black")





hmd_method_pearson = numpy.zeros((4, 10))
hmd_method_concordance = numpy.zeros((4, 10))


x_labels = []
y_labels = []
for i, pns in enumerate(pipeline_name_short):
    if pns == "mg":
        continue
    j = -1
    x_labels = []
    for s in samples:
        for r in replicates:
            j += 1
            x_labels.append(f"{s}_{r}")
            # print(pns, s, r)
            hmd_method_pearson[i, j] = method_comparison_data_pearson[pns][(s, r)]
            hmd_method_concordance[i, j] = method_comparison_data_concordance[pns][(s, r)]


for i in range(4):
    pns = pipeline_name_short[i]
    yl = f"{pipeline_name_short[i]} vs mG"
    y_labels.append(yl)

ax3.imshow(hmd_method_pearson, cmap="Reds", vmin=0.6, vmax=1)
ax3.set_title("Method Pearson Correlation")
ax3.set_xticks(numpy.arange(len(x_labels)))
ax3.set_xticklabels(x_labels, rotation=90)
ax3.set_yticks(numpy.arange(4))
ax3.set_yticklabels(y_labels)
ax3.grid(False)

ax4.imshow(hmd_method_concordance, cmap="Reds", vmin=0.6, vmax=1)
ax4.set_title("Method Concordance")
ax4.set_xticks(numpy.arange(len(x_labels)))
ax4.set_xticklabels(x_labels, rotation=90)
ax4.set_yticks(numpy.arange(4))
ax4.set_yticklabels(y_labels)
ax4.grid(False)



for i in range(len(pipeline_name_short)):
    if i == 4:
        continue
    for j in range(len(x_labels)):
        ax3.text(j, i, f"{hmd_method_pearson[i, j]:.2f}", ha="center", va="center", color="white")
        ax4.text(j, i, f"{hmd_method_concordance[i, j]:.2f}", ha="center", va="center", color="white")



# make color scale for ax5
xxx = numpy.zeros((1, 5))
xxx[0, 0] = 0.6
xxx[0, 1] = 0.7
xxx[0, 2] = 0.8
xxx[0, 3] = 0.9
xxx[0, 4] = 1

# add color bar
f5 = ax5.imshow(xxx, cmap="Reds", vmin=0.6, vmax=1)
plt.colorbar(ax5.imshow(xxx, cmap="Reds", vmin=0.6, vmax=1), ax=ax5, orientation="vertical")



fig4XXX.savefig("./figures/S13.pdf", dpi=300, bbox_inches="tight")





In [ ]:
# PCA Plot

cpg_pca_data = "4D.PCA.1.tsv"





bismark_cpg_data = {}
methylgrapher_lifted_cytosine_data = {}


for s in samples:
    bismark_cpg_data[s] = {}
    methylgrapher_lifted_cytosine_data[s] = {}


    for r in replicates:
        print(s, r)


        bismark_cpg_data[s][r] = {}
        methylgrapher_lifted_cytosine_data[s][r] = {}

        mg2l_fp = f"/scratch/wzhang/projects/ggWGBS/methylgrapher_no_trim/surjection/{s}_{r}.methylc.gz"
        bs_fp = f"/scratch/wzhang/projects/ggWGBS/data/bismark_hg38/{s}_{r}.cpg.methylc"

        for l in open(bs_fp):
            l = l.strip().split("\t")
            chrom, start, end, context, ml, strand, cov = l

            start = int(start)
            met = int(round(float(ml) * int(cov)))
            cov = int(cov)

            if chrom not in bismark_cpg_data[s][r]:
                bismark_cpg_data[s][r][chrom] = {}

            bismark_cpg_data[s][r][chrom][start] = (met, cov)

        for l in gzip.open(mg2l_fp, "rt"):
            l = l.strip().split("\t")
            chrom, start, end, context, ml, strand, cov = l

            start = int(start)
            met = int(round(float(ml) * int(cov)))
            cov = int(cov)

            if chrom not in methylgrapher_lifted_cytosine_data[s][r]:
                methylgrapher_lifted_cytosine_data[s][r][chrom] = {}

            methylgrapher_lifted_cytosine_data[s][r][chrom][start] = (met, cov)

cpg_pca_data_fh = open(cpg_pca_data, "w")
cpg_ref_fp = "/scratch/wzhang/ref/linear/hg38/bismark/cpg_sites.bed"
for l in open(cpg_ref_fp):
    l = l.strip().split("\t")
    chrom, start, end = l
    start = int(start) -1
    
    pass_cov = True

    bs_cpgs = []
    mg_cpgs = []

    for s in samples:
        for r in replicates:
            if chrom not in bismark_cpg_data[s][r] or chrom not in methylgrapher_lifted_cytosine_data[s][r]:
                pass_cov = False
                break
                
            met, cov = bismark_cpg_data[s][r][chrom].get(start, (0, 0))
            if cov <= min_cov:
                pass_cov = False
                break
            bs_cpgs.append(met/cov)


            m1, c1 = methylgrapher_lifted_cytosine_data[s][r][chrom].get(start, (0, 0))
            m2, c2 = methylgrapher_lifted_cytosine_data[s][r][chrom].get(start+1, (0, 0))
            met = m1 + m2
            cov = c1 + c2
            if cov <= min_cov:
                pass_cov = False
                break
            mg_cpgs.append(met/cov)


    if not pass_cov:
        continue



    line = [chrom, start, end] + bs_cpgs + mg_cpgs
    line_str = "\t".join(map(str, line)) + "\n"
    cpg_pca_data_fh.write(line_str)
    
cpg_pca_data_fh.close()






In [ ]:
# https://hgdownload.cse.ucsc.edu/goldenpath/hg38/bigZips/hg38.chrom.sizes

genome_size_str = """chr1	248956422
chr2	242193529
chr3	198295559
chr4	190214555
chr5	181538259
chr6	170805979
chr7	159345973
chrX	156040895
chr8	145138636
chr9	138394717
chr11	135086622
chr10	133797422
chr12	133275309
chr13	114364328
chr14	107043718
chr15	101991189
chr16	90338345
chr17	83257441
chr18	80373285
chr20	64444167
chr19	58617616
chrY	57227415
chr22	50818468
chr21	46709983"""

genome_size = {}
for l in genome_size_str.split("\n"):
    l = l.strip().split("\t")
    genome_size[l[0]] = int(l[1])


genome_bins = {}
bin_size = 3000
for chrom in sorted(genome_size.keys(), key=lambda x: int(x[3:] if x[3:] != "X" and x[3:] != "Y" else "23" if x[3:] == "M" else "24")):
    size = genome_size[chrom]
    
    start, end = 0, bin_size
    
    while end < size:
        genome_bins[bin_id] = (chrom, start, end)
        start += bin_size
        end += bin_size
        
        bin_id = f"{chrom}_{start}"
    print(bin_id)
    

def get_bin_id(chrom, start):
    res = f"{chrom}_{start//bin_size*bin_size}"
    if res not in genome_bins:
        if len(chrom) > 5:
            print(chrom, start, None)
        return None
    return res


genome_bins_data = {}
for l in open(cpg_pca_data):
    l = l.strip().split("\t")
    
    chrom, start, end = l[:3]
    values = list(map(float, l[3:]))
    
    start = int(start)
    end = int(end)
    
    # print(chrom, start, end, len(values))
    
    bin_id = get_bin_id(chrom, start)
    if bin_id == None:
        continue
    
    if bin_id not in genome_bins_data:
        genome_bins_data[bin_id] = []
    
    genome_bins_data[bin_id].append(values)
    

In [ ]:
# Use Per-CpG 


common_cpg_data = {}
for l in open(cpg_pca_data):
    l = l.strip().split("\t")
    
    chrom, start, end = l[:3]
    values = list(map(float, l[3:]))
    
    ave = sum(values) / len(values)
    all_the_same = True
    for v in values:
        if abs(v - ave) > 0.00001:
            all_the_same = False
    if all_the_same:
        # print(chrom, start, "all the same, ", values)
        continue
    
    cpg_id = f"{chrom}_{start}"
    common_cpg_data[cpg_id] = values

cpg_ids = list(common_cpg_data.keys())
    


bin_pca_data_fh = open("4D.PCA.2.CpG.tsv", "w")
i = 0
for m in ["Bismark", "methylGrapher"]:
    for s in samples:
        for r in replicates:
            line = [s, r, m]
            
            for cpg_id in cpg_ids:
                line.append(common_cpg_data[cpg_id][i])
            
            line_str = "\t".join(list(map(str, line))) + "\n"
            bin_pca_data_fh.write(line_str)
            
            
            i += 1
bin_pca_data_fh.close()



In [ ]:
# Use Binned genome
bin_pca_data = "4D.PCA.2.tsv"

genome_bins_data_averaged = {}
for bin_id in genome_bins_data:
    
    
    sum_bin_ml = [0] * 20
    cpg_count = len(genome_bins_data[bin_id])
    
    for each_cpg in genome_bins_data[bin_id]:
        
        for i in range(20):
            sum_bin_ml[i] += each_cpg[i]
    
    for i in range(20):
        sum_bin_ml[i] /= cpg_count
    
    if cpg_count < 5:
        pass
        # print(bin_id, f"cpg count ({cpg_count}) < 10")
        #continue
    
    ave = sum(sum_bin_ml) / 20
    all_the_same = True
    for i in range(20):
        if abs(sum_bin_ml[i] - ave) > 0.0001:
            all_the_same = False
    if all_the_same:
        print(bin_id, f"all the same, {sum_bin_ml}")
        continue
    
    
    genome_bins_data_averaged[bin_id] = sum_bin_ml


bin_ids = list(genome_bins_data_averaged.keys())


bin_pca_data_fh = open(bin_pca_data, "w")
i = 0
for m in ["Bismark", "methylGrapher"]:
    for s in samples:
        for r in replicates:
            line = [s, r, m]
            
            for bin_id in bin_ids:
                line.append(genome_bins_data_averaged[bin_id][i])
            
            line_str = "\t".join(list(map(str, line))) + "\n"
            bin_pca_data_fh.write(line_str)
            
            
            i += 1
bin_pca_data_fh.close()
            



    

In [ ]:




df = pandas.read_csv("4D.PCA.2.CpG.tsv", sep="\t", header=None)
# print(df.shape)
# print(df.head())


labels = {}

for i in range(20):
    s = df.iloc[i, 0]
    r = df.iloc[i, 1]
    m = df.iloc[i, 2]
    labels[i] = f"{s}_{r}_{m}"

# print()

df2 = df.iloc[:, 3:]
# print(df2.shape)






pca = PCA(n_components=2)

principalComponents = pca.fit_transform(df2)


# print(principalComponents.shape)
# print(principalComponents)


for i in range(principalComponents.shape[0]):
    label = labels[i]
    pc1 = principalComponents[i, 0]
    pc2 = principalComponents[i, 1]
    
    
    # print(label, pc1, pc2)
    





In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
# ax.scatter(principalComponents[:, 0], principalComponents[:, 1])

sample_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
# replicate_marker = ["." , "," , "o" , "v" , "^" , "<", ">"]

markers = ["," , "^" ]

pca_data = {}
for i in range(principalComponents.shape[0]):
    label = labels[i]
    s, r, m = label.split("_")
    if s not in pca_data:
        pca_data[s] = {}
    if r not in pca_data[s]:
        pca_data[s][r] = {}
    pca_data[s][r][m] = (principalComponents[i, 0], principalComponents[i, 1])


for s in sorted(pca_data.keys()):
    for r in sorted(pca_data[s].keys()):
        for m in pca_data[s][r]:
            
            new_m = "Bis"
            if m == "methylGrapher":
                new_m = "mG"
            new_label = f"{s}_{r} ({new_m})"
            pc1 = pca_data[s][r][m][0]
            pc2 = pca_data[s][r][m][1]
            
            color = sample_colors[samples.index(s)]
            marker = markers[replicates.index(int(r))]
            alpha = 1
            
            if m != "Bismark":
                # marker = markers[replicates.index(int(r))+2]
                alpha = 0.5
            
            ax.scatter([pc1], [pc2], color=color, label=new_label, marker=marker, alpha=alpha, s=60, facecolors='none')
            
            ax.set_xlabel(f'PC 1, Variance ({round(pca.explained_variance_ratio_[0]*100, 2)}%)')
            ax.set_ylabel(f'PC 2, Variance ({round(pca.explained_variance_ratio_[1]*100, 2)}%)')
            ax.set_title(f'Common CpG Site Methylation PCA (N = {df.shape[1]/1e6:0.2f}M)')

ax.legend()
plt.savefig("./figures/4D.PCA.CpG.pdf", bbox_inches='tight')


In [ ]:
# Figure 5A

#bismark_cpg_data = {}
#methylgrapher_cytosine_data = {}

pan_cpg_list = []
pan_cpg_list_fp = f"/scratch/wzhang/projects/ggWGBS/cpg_categorization/pan_v1_cpg.tsv"
hg38_loci_regex = re.compile(r"hg38\((.*):(\d*)\)")


seen = set()
with open(pan_cpg_list_fp) as fh:
    for l in fh:
        l = l.strip().split("\t")
        cpgid, segID1, seg_pos1, segID2, seg_pos2, cpg_type = l
        seg_pos1 = int(seg_pos1)
        seg_pos2 = int(seg_pos2)

        info = tuple([cpg_type])
        if cpg_type.startswith("hg38"):
            chrom, start = hg38_loci_regex.findall(cpg_type)[0]
            start = int(start)
            info = ("hg38", chrom, start)
            # print(cpg_type, chrom, start)
            # break

        xxx = info[0]
        if xxx not in seen:
            seen.add(xxx)
            print(xxx, l)

        c = (segID1, seg_pos1, segID2, seg_pos2, info)
        pan_cpg_list.append(c)
        



In [ ]:

cpg_unique_data = {}

for s in samples:
    cpg_unique_data[s] = {}

    for r in replicates:
        print(s, r)
        bs_cpg = bismark_cpg_data[s][r]
        mg_cytosine = methylgrapher_cytosine_data[s][r]

        total_bs = 0
        for chrom in bs_cpg.keys():
            total_bs += len(bs_cpg[chrom])

        common, bss = 0, 0
        mg_counts = [0, 0, 0]
        for cpg in pan_cpg_list:
            segID1, seg_pos1, segID2, seg_pos2, info = cpg

            mge, bse = False, False
            if segID1 in mg_cytosine:
                if seg_pos1 in mg_cytosine[segID1]:
                    mge = True
            if segID2 in mg_cytosine:
                if seg_pos2 in mg_cytosine[segID2]:
                    mge = True

            cpg_type = info[0]
            if cpg_type == "hg38":
                chrom = info[1]
                start = info[2]
                
                if chrom in bs_cpg:
                    if start in bs_cpg[chrom]:
                        bse = True

            if mge and bse:
                common += 1
            elif bse:
                bss += 1
            elif mge:
                i = ["hg38", "SNV", "SV"].index(cpg_type)
                #if i == 0 and len(chrom) > 5:
                #     i = 3
                mg_counts[i] += 1
                
        cpg_unique_data[s][r] = [bss, common] + mg_counts


        
        print(common)
        print(mg_counts, sum(mg_counts))
        print(bss)
        print()




In [ ]:

fig5A, axes5A = plt.subplots(10, 1, figsize=(7, 28))

axes_i = 0
for s in samples:

    for r in replicates:
        d = cpg_unique_data[s][r][:]

        bar_x = ["Bis Unique", "Common", "mG within hg38", "mG SNV", "mG Others"]

        bar_x = list(reversed(bar_x))
        d = list(reversed(d))
        
        axes5A[axes_i].barh(bar_x, d, color="#88a2d9", height=0.6)

        axes5A[axes_i].set_title(f"{s}_{r}")
        # axes5A[axes_i].legend()

        xticks = [0, 1e6, 2e6]
        xticks_label = [0, 1, 2]
        for i in range(5):
            m = 0.5*1e7*(i+1)
            label = str(round(5*(i+1)))
            
            xticks.append(m)
            xticks_label.append(label)
        
        axes5A[axes_i].set_xticks(xticks, xticks_label)

        axes_i += 1

fig5A.savefig(f"./figures/5A.pdf", bbox_inches='tight')



In [ ]:

for r in range(1, 7):
    bs_hg38_url = f"/scratch/wzhang/projects/ggWGBS/entex_dataset/bismark/ENC001/{r}/hg38/track.cpg.methylc.gz"
    mg_url = f"/scratch/wzhang/projects/ggWGBS/methylgrapher_no_trim/ENC001/{r}/default.graph.CG.merged.tsv"

    c1, c2 = 0, 0
    with gzip.open(bs_hg38_url, "rt") as fh:
        for l in fh:
            l = l.strip().split("\t")
            cov = int(l[-1])
            
            if cov > 5:
                c1 += 1
            c2 +=1
            
    c3, c4 = 0, 0
    with open(mg_url) as fh:
        for l in fh:
            l = l.strip().split("\t")
            cov = int(l[-1])
            
            if cov > 5:
                c3 += 1
            c4 +=1
    
    print(r, c1, c2, c3, c4)


In [ ]:
figure6a_data_str = """
1 14616592 26865902 16429150 29074756
2 15101570 26850694 17048709 29110264
3 16563087 27364276 18779790 29703975
4 13383780 26209787 16003132 28722950
5 14706209 27334725 19019548 29989821
6 11939444 25750073 14442447 28161547
""".strip().split("\n")




fig, ax = plt.subplots(figsize=(5, 6))


bismark_cpg_counts = []
bismark_cpg_counts_cov5 = []
methylgrapher_cpg_counts = []
methylgrapher_cpg_counts_cov5 = []

for l in figure6a_data_str:
    l = l.strip().split(" ")
    for i in range(0, 5):
        l[i] = int(l[i])

    bismark_cpg_counts.append(l[2])
    bismark_cpg_counts_cov5.append(l[1])

    methylgrapher_cpg_counts.append(l[4])
    methylgrapher_cpg_counts_cov5.append(l[3])

    d = l[4] - l[2]
    print(d, d/l[2])


label = []
bismark_x = []
methylgrapher_x = []


for i in range(6):

    l = individual_experiment_tissue_label_short["ENC001"][i+1]
    lc = l[0].upper() + l[1:]
    label.append(lc)
    bismark_x.append(i-0.15)
    methylgrapher_x.append(i+0.15)



ax.bar(bismark_x, bismark_cpg_counts, width=0.3, color=pipeline_color_lookup["bs"], label="Bismark (0<Cov<=5)", alpha=0.5)
ax.bar(methylgrapher_x, methylgrapher_cpg_counts, width=0.3, color=pipeline_color_lookup["mg"], label="methylGrapher  (0<Cov<=5)", alpha=0.5)

ax.bar(bismark_x, bismark_cpg_counts_cov5, width=0.3, color=pipeline_color_lookup["bs"], label="Bismark (Cov>5)")
ax.bar(methylgrapher_x, methylgrapher_cpg_counts_cov5, width=0.3, color=pipeline_color_lookup["mg"], label="methylGrapher (Cov>5)")

ax.set_xticks(range(6), label)
ax.set_title("# of CpG Site")

ax.legend(loc="lower right")

# rotate x label
plt.xticks(rotation=90)


ax.set_yticks([0, 1e7, 2e7, 3e7], ["", "10M", "20M", "30M"])

plt.savefig("./figures/6A.pdf", bbox_inches='tight')







In [ ]:
# Divider - EN-TEx sample ENC001 analysis from now on

In [ ]:
enc_001_long_read_vcf_fp = f"/scratch/wzhang/projects/ggWGBS/genome_compare/vcf_analysis/ont.vcf"

ont_vcf_info = {}
for i, l in enumerate(open(enc_001_long_read_vcf_fp)):
    if l.startswith("#"):
        continue

    l = l.strip().split("\t")
    l[1] = int(l[1])

    ont_vcf_info[i] = l


In [ ]:

cpg_count = 0
sv_lengths = []

for ln, l in enumerate(open(enc_001_long_read_vcf_fp)):
    if l.startswith("#"):
        continue

    l = l.strip().split("\t")

    chrom = l[0]
    pos = int(l[1])
    ref = l[3].upper()
    alt = l[4].upper()

    cpg_count += alt.upper().count("CG")


    if len(ref) != 1:
        continue

    if len(alt) <= 50:
        continue

    sv_lengths.append(len(alt))

print(len(sv_lengths))
print(sum(sv_lengths)/1e6)
print(cpg_count)




In [ ]:

figS16, axesS16 = plt.subplots(2, 1, figsize=(6, 13))
plt.subplots_adjust(hspace=0.3)




sv_perfect_match = []

x = []
y = []
data2 = {}
for ipc in [0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.985, 0.99, 0.992, 0.994, 0.996, 0.998, 0.999, 1]:
    data2[ipc] = [[]]



with open("/scratch/wzhang/projects/ggWGBS/genome_compare/vcf_analysis/flanking_mod/ins_50_flank_1000.graphalign.stat.tsv") as fh:

    for l in fh:
        l = l.strip().split("\t")
        svl = int(l[1])
        ip = float(l[2])
        # print(svl, ip)

        x.append(svl)
        y.append(ip)

        if ip >= 1:
            sv_perfect_match.append(svl)

        for ipc in data2.keys():
            if ip >= ipc:
                data2[ipc][0].append(svl)



x = list(sorted(data2.keys()))
y1 = [len(data2[ipc][0]) for ipc in x]
y2 = [sum(data2[ipc][0])/1e6 for ipc in x]


axesS16[1].scatter(x, y1, s=1)
axesS16[1].plot(x, y1)
axesS16[1].set_title("#common insertions between pangenome and ENC001")
axesS16[1].set_xlabel("Identity percentage threshold")
axesS16[1].set_ylabel("# of insertions")
axesS16[1].set_ylim(0, 10000)
axesS16[1].set_yticks(range(0, 10001, 2000))


axesS16[0].scatter(x, y2, s=1)
axesS16[0].plot(x, y2)
axesS16[0].set_title("Total length common insertion between pangenome and ENC001")
axesS16[0].set_xlabel("Identity percentage threshold")
axesS16[0].set_ylabel("Total Insertion length (Mb)")



figS16.savefig("./figures/S16.pdf", bbox_inches='tight')


In [ ]:

flanking_length = 1000

p_graph_fp = f"/scratch/wzhang/projects/ggWGBS/genome_compare/vcf_analysis/flanking_mod/ins_50_flank_{flanking_length}.graphalign.stat.tsv"
p_h1_fp = f"/scratch/wzhang/projects/ggWGBS/genome_compare/vcf_analysis/flanking_mod/ins_50_flank_{flanking_length}.hap1.minimap2.stat.tsv"
p_h2_fp = f"/scratch/wzhang/projects/ggWGBS/genome_compare/vcf_analysis/flanking_mod/ins_50_flank_{flanking_length}.hap2.minimap2.stat.tsv"


graph_sv_lns = set()
h1_sv_lns = set()
h2_sv_lns = set()
with open(p_graph_fp) as fh:
    for l in fh:
        l = l.strip().split()
        ln = int(l[0].split(":")[0])
        sv_length = int(l[1])
        ip = float(l[2])
        if ip < 0.9:
            continue

        graph_sv_lns.add(ln)
        # print(ln, sv_length, ip)


with open(p_h1_fp) as fh:
    for l in fh:
        l = l.strip().split()
        ln = int(l[0].split(":")[0])
        sv_length = int(l[1])
        #ip = float(l[2])

        if ln in graph_sv_lns:
            h1_sv_lns.add(ln)

with open(p_h2_fp) as fh:
    for l in fh:
        l = l.strip().split()
        ln = int(l[0].split(":")[0])
        sv_length = int(l[1])
        #ip = float(l[2])

        if ln in graph_sv_lns:
            h2_sv_lns.add(ln)


graph_sv_stat = [0, 0, 0]
h1_sv_stat    = [0, 0, 0]
h2_sv_stat    = [0, 0, 0]
for ln in graph_sv_lns:

    sv_seq = ont_vcf_info[ln][4].upper()
    cg_count = sv_seq.count("CG")
    svl = len(sv_seq)

    graph_sv_stat[0] += 1
    graph_sv_stat[1] += svl
    graph_sv_stat[2] += cg_count


for ln in h1_sv_lns:

    sv_seq = ont_vcf_info[ln][4].upper()
    cg_count = sv_seq.count("CG")
    svl = len(sv_seq)

    h1_sv_stat[0] += 1
    h1_sv_stat[1] += svl
    h1_sv_stat[2] += cg_count

for ln in h2_sv_lns:

    sv_seq = ont_vcf_info[ln][4].upper()
    cg_count = sv_seq.count("CG")
    svl = len(sv_seq)

    h2_sv_stat[0] += 1
    h2_sv_stat[1] += svl
    h2_sv_stat[2] += cg_count

graph_sv_stat[1] /= 1e6
h1_sv_stat[1] /= 1e6
h2_sv_stat[1] /= 1e6

print(graph_sv_stat)
print(h1_sv_stat)
print(h2_sv_stat)


In [ ]:




flanking_length = 1000
aligner = ["graphalign", "minigraph", "vg"][0]


cpg_count = 0
sv_count = 0
sv_length_total = 0



def methylc_reader(methylc_fp):
    res = {}
    with gzip.open(methylc_fp, "rt") as f:
        for l in f:
            l = l.strip().split("\t")
            chrom, start, end, context, ml, strand, cov = l

            start = int(start)
            end = int(end)
            ml = float(ml)
            cov = int(cov)

            if chrom not in res:
                res[chrom] = {}
            if start not in res[chrom]:
                res[chrom][start] = (ml, cov)

    return res





processed_fp = f"/scratch/wzhang/projects/ggWGBS/genome_compare/vcf_analysis/flanking_mod/ins_50_flank_{flanking_length}.{aligner}.stat.tsv"
sv2graph = {}
with open(processed_fp) as fh:
    for l in fh:
        l = l.strip().split("\t")
        qn = l[0]
        sv_length = int(l[1])
        ip = float(l[2])
        if ip < 0.9:
            continue

        coordinate_lifting_sv2graph = json.loads(l[4])

        sv2graph[int(qn.split(":")[0])] = coordinate_lifting_sv2graph

        alt_seq = ont_vcf_info[int(qn.split(":")[0])][4].upper()

        cpg_count += alt_seq.count("CG")
        sv_count += 1
        sv_length_total += len(alt_seq)



sv2ind = {}
for hi in [1, 2]:
    processed_fp = f"/scratch/wzhang/projects/ggWGBS/genome_compare/vcf_analysis/flanking_mod/ins_50_flank_{flanking_length}.hap{hi}.minimap2.stat.tsv"

    sv2ind[hi] = {}
    with open(processed_fp) as fh:
        for l in fh:
            l = l.strip().split("\t")
            qn = l[0]
            sv2ind[hi][int(qn.split(":")[0])] = json.loads(l[3])


sv_region = {}
for hi in [1, 2]:
    insertion_region_fp = f"/scratch/wzhang/projects/ggWGBS/genome_compare/vcf_analysis/flanking_mod/ins_50_flank_{flanking_length}.hap{hi}.minimap2.insertion.bed"
    tl = 0
    with open(insertion_region_fp) as fh:
        for l in fh:
            l = l.strip().split()
            chrom, start, end = l
            start = int(start)
            end = int(end)
            if chrom not in sv_region:
                sv_region[chrom] = []
            sv_region[chrom].append((start, end))
            tl += end - start







individual = f"ENC001"

for experiment_i in range(1, 7):

    for hi in range(1, 3):

        bs_hap = f"/scratch/wzhang/projects/ggWGBS/entex_dataset/bismark/{individual}/{experiment_i}/hap{hi}/track.cpg.methylc.gz"
        bs_hap_data = methylc_reader(bs_hap)

        bs_cpg_count = 0
        for chrom in sv_region.keys():
            for start, end in sv_region[chrom]:
                for pos in range(start, end+1):
                    if pos not in bs_hap_data.get(chrom, {}):
                        continue
                    bs_cpg_count += 1

        mgl_hap = f"/scratch/wzhang/projects/ggWGBS/methylgrapher_no_trim/{individual}/{experiment_i}/hap{hi}.cpg.methylc.gz"
        mgl_hap_data = methylc_reader(mgl_hap)

        mgl_cpg_count = 0
        for chrom in sv_region.keys():
            for start, end in sv_region[chrom]:
                for pos in range(start, end+1):
                    if pos not in mgl_hap_data.get(chrom, {}):
                        continue
                    mgl_cpg_count += 1

        print(individual, experiment_i, hi, bs_cpg_count, mgl_cpg_count)




In [ ]:




fig, axes = plt.subplots(6, 2, figsize=(8, 26))
plt.subplots_adjust(hspace=0.3)

def methylc_reader(methylc_fp):
    res = {}
    with gzip.open(methylc_fp, "rt") as f:
        for l in f:
            l = l.strip().split("\t")
            chrom, start, end, context, ml, strand, cov = l

            start = int(start)
            end = int(end)
            ml = float(ml)
            cov = int(cov)

            if chrom not in res:
                res[chrom] = {}
            if start not in res[chrom]:
                res[chrom][start] = (ml, cov)

    return res

individual = f"ENC001"
for experiment_i in range(1, 7):

    tissue_name = individual_experiment_tissue_label_short[individual].get(experiment_i, "Unknown")
    # Capitalize tissue name
    tissue_name = tissue_name[0].upper() + tissue_name[1:]

    for hi in range(1, 3):
        ax = axes[experiment_i-1][hi-1]
        bs_hap = f"/scratch/wzhang/projects/ggWGBS/entex_dataset/bismark/{individual}/{experiment_i}/hap{hi}/track.cpg.methylc.gz"
        lifted = f"/scratch/wzhang/projects/ggWGBS/methylgrapher_no_trim/ENC001/{experiment_i}/hap{hi}.cpg.methylc.gz"

        bs_hap_methylc = methylc_reader(bs_hap)
        lifted_methylc = methylc_reader(lifted)

        cpg_count = 0
        x = []
        y = []
        for chrom in bs_hap_methylc:
            if chrom not in lifted_methylc:
                continue

            for start in bs_hap_methylc[chrom]:
                bs_ml, bs_cov = bs_hap_methylc[chrom][start]
                lifted_met, lifted_cov = 0, 0


                lifted_d0 = lifted_methylc[chrom].get(start, (0, 0))
                lifted_ml = lifted_d0[0]
                lifted_cov = lifted_d0[1]

                if lifted_cov == 0:
                    continue

                cpg_count += 1
                if bs_cov <= 5 or lifted_cov <= 5:
                    continue

                x.append(bs_ml)
                y.append(lifted_ml)


        k, b, r, p, std_err = scipy.stats.linregress(x, y)


        concordance_interval = 0.15
        concordance = 0
        for i in range(len(x)):
            x0 = x[i]
            y0 = y[i]

            cflag = abs(x0 - y0) <= concordance_interval
            if cflag:
                concordance += 1





        ax.scatter(x, y, s=0.8)
        ax.plot([0, 1], [b, k+b], color="green")
        ax.text(0.08, 0.75, f"y = {k:.2f}x + {b:.2f}\nr = {r:.2f}\nN = {len(x)}", transform=ax.transAxes)

        ax.set_title(f"{tissue_name} hap{hi}")
        ax.set_xlabel("Methylation (Bismark)")
        ax.set_ylabel("Methylation (methylGrapher)")

        print(individual, experiment_i, hi, cpg_count, len(x), k, b, r, concordance/len(x))


        fig.savefig(f"./figures/S17.pdf", dpi=300, bbox_inches='tight')








